In [ ]:
import sys
import os
import xarray as xr
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import dask.array as da

from datetime import datetime, timedelta
import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import warnings
import psutil
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from matplotlib import patches
from matplotlib import cm

from metpy import calc as mpcalc
from metpy.units import units

In [ ]:
os.chdir('/w2-data2/willytsai/python_module/')
import SCAI_calc4obj as scai 

In [ ]:
def subset_TRMM(file_path,time_cent,lat_cent,lon_cent,days=3,lat_inc=5,lon_inc=5):

    os.chdir(file_path)
    
    n=0
    # daily file
    date_sel = [time_cent + timedelta(days=day) for day in range(-days,days+1)] # selected files
    for i,date in enumerate(date_sel):
        date_str = datetime.strftime(date,format='%Y%m%d')
        files = sorted(glob('*'+date_str+'*')) # sub-daily
        for file in files:
            data = xr.open_dataset(file)
            data = data.sel(nlat=slice(lat_cent-lat_inc,lat_cent+lat_inc),
                            nlon=slice(lon_cent-lon_inc,lon_cent+lon_inc))
            data = data.assign_coords(time=datetime.strptime(file[5:16],'%Y%m%d.%H'))
        
            if n == 0: 
                tmp = data; n+=1
            else:
                tmp = xr.concat([tmp,data],dim='time')
    
    # extract fetched time window 
    data_sub = tmp.sel(time=slice(time_cent-timedelta(days=days),time_cent+timedelta(days=days)))
    
    return data_sub

In [ ]:
def subset_MERRA2(file_path,time_cent,lat_cent,lon_cent,days=3,lat_inc=5,lon_inc=5):

    os.chdir(file_path)
    
    n=0
    # daily file 
    date_sel = [time_cent + timedelta(days=day) for day in range(-days,days+1)] # selected files
    for i,date in enumerate(date_sel):
        #print(date)
        date_str = datetime.strftime(date,format='%Y%m%d')
        file = sorted(glob('*'+date_str+'*'))[0]
        data = xr.open_dataset(file)
        data = data.sel(lat=slice(lat_cent-lat_inc,lat_cent+lat_inc),
                        lon=slice(lon_cent-lon_inc,lon_cent+lon_inc))
        
        if n == 0: 
            tmp = data; n+=1
        else:
            tmp = xr.concat([tmp,data],'time')
    
    # extract fetched time window 
    data_sub = tmp.sel(time=slice(time_cent-timedelta(days=days),time_cent+timedelta(days=days)))
    
    return data_sub

In [ ]:
def subset_gridsat(file_path,time_cent,lat_cent,lon_cent,days=3,lat_inc=5,lon_inc=5):

    os.chdir(file_path)
    
    n=0
    # daily file
    date_sel = [time_cent + timedelta(days=day) for day in range(-days,days+1)] # selected files
    for i,date in enumerate(date_sel):
        date_str = datetime.strftime(date,format='%Y.%m.%d')
        files = sorted(glob('*'+date_str+'*')) # sub-daily
        for file in files:
            data = xr.open_dataset(file)
            data = data.sel(lat=slice(lat_cent-lat_inc,lat_cent+lat_inc)
                            ,lon=slice(lon_cent-lon_inc,lon_cent+lon_inc))
            data = data.assign_coords(time=datetime.strptime(file[11:24],'%Y.%m.%d.%H'))
        
            if n == 0: 
                tmp = data; n+=1
            else:
                tmp = xr.concat([tmp,data],'time')
    
    # extract fetched time window 
    data_sub = tmp.sel(time=slice(time_cent-timedelta(days=days),time_cent+timedelta(days=days)))
    
    return data_sub

In [ ]:
def SCAI_index(prec_sm,bt_sm):

    N_psT = np.zeros(bt_sm.shape[0]) # BT_based
    N_prec = np.zeros(prec_sm.shape[0]) # prec_based
    SCAI_BT = np.zeros(bt_sm.shape[0]) # cldfrac over domain by BT criteria
    SCAI_p = np.zeros(prec_sm.shape[0]) # cldfrac over domain by prec criteria
    
   # N and SCAI indices
    for t in range(bt_sm.shape[0]):
        label_obj, N = scai.labeled_obj(bt_sm[t,:,:].values,cri=240,flag=2) # less than 240 W/m^2
        SCAI_BT[t] = scai.SCAI_calc(label_obj,N,dx=0.25*100000,L=10*100000)
        N_psT[t] = N
    
    for t in range(prec_sm.shape[0]):
        label_obj, N = scai.labeled_obj(prec_sm[t,:,:].values,cri=1,flag=1) # larger than 1mm/day
        SCAI_p[t] = scai.SCAI_calc(label_obj,N,dx=0.25*100000,L=10*100000)    
        N_prec[t] = N

    return (SCAI_BT,SCAI_p,N_psT,N_prec)

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
ISCCP_dir = '/w2-data/willytsai/ISCCP/hgg/'
TRMM_dir = '/w2-data2/willytsai/TRMM_3hr/TRMM/'
MERRA2_dir = '/w2-data2/willytsai/MERRA2/'
MERRA2_3d_dir = '/w2-data/willytsai/'
ERA5_dir = '/w2-data/willytsai/ERA5/'
Gridsat_dir = '/w2-data/willytsai/gridsat_BT/remap_0.25deg/'
fig_dir = '/w2-data/willytsai/ISCCP/fig/'

### Read data_event from written .dat files

In [ ]:
# load data
os.chdir('/w2-data/willytsai/ISCCP/')

# variable namelist
var_name = np.loadtxt('var_name_2014_2018.dat',dtype='U16')
# event time
var_time = np.loadtxt('time_event_2014_2018.dat',dtype='U13')
time_event = []
for t in var_time:
    tmp = datetime.strptime(t,'%Y-%m-%d-%H')
    time_event.append(tmp)

var_dataset = (np.loadtxt('var_event_2014_2018.dat'))

In [ ]:
os.chdir('/w2-data/willytsai/TQUV_3hr/')
files = glob('*.nc')
lev = xr.open_dataset(files[0]).T.lev # MERRA2 standard level 
del files
os.chdir('/w2-data/willytsai/Cloudfrac_3hr/')
lev_m = np.loadtxt('model_lev.txt')[:,1]# MERRA2 model level, 58 levs [72-15]

In [ ]:
def event_region(lat_event,lon_event,lon_w,lon_e,lat_s,lat_n):
    'select specfic region'
    idx = np.where(np.logical_and(lon_event >= lon_w, lon_event <= lon_e))[0]
    idy = np.where(np.logical_and(lat_event >= lat_s, lat_event <= lat_n))[0]
    idc = np.intersect1d(idx,idy)
    return idc

In [ ]:
lat_event = var_dataset[:,0]
lon_event = var_dataset[:,1]

In [ ]:
time_event = np.asarray(time_event)
lat_event = lat_event
lon_event = lon_event

In [ ]:
os.chdir('/w2-data/willytsai/ISCCP')
T_all = np.load('T_all.mat.npy')
q_all = np.load('q_all.mat.npy')
u_all = np.load('u_all.mat.npy')
v_all = np.load('v_all.mat.npy')
N_all = np.load('N_all.mat.npy')
SCAIbt_all = np.load('SCAIbt_all.mat.npy')
bt_all = np.load('bt_all.mat.npy')
llws_all = np.load('llws_all.mat.npy')
prec_all = np.load('prec_all.mat.npy')
cwv_all = np.load('cwv_all.mat.npy')
crh_all = np.load('crh_all.mat.npy')

omega_all = np.load('omega_all.mat.npy')
rh_all = np.load('rh_all.mat.npy')
cwvstd_all = np.load('cwvstd_all.mat.npy')
SF_all = np.load('SF_all.mat.npy')
cldfrac_all = np.load('cldfrac_all.mat.npy')
CFv_all = np.load('CFv_all.mat.npy')
div_all  = np.load('div_all.mat.npy')
cwvten_dyn_all = np.load('cwvten_dyn_all.mat.npy')
cwvten_phy_all = np.load('cwvten_phy_all.mat.npy')
cwvten_ana_all = np.load('cwvten_ana_all.mat.npy')
cwvten_tot_all = cwvten_dyn_all+cwvten_phy_all+cwvten_ana_all
Abar_all = (cldfrac_all/N_all*1600*(25*25))**0.5 # square root of Area [km]
cape_all = np.load('cape_all.mat.npy')
cin_all = np.load('cin_all.mat.npy')

cldamt_cu_all = np.load('cldamt_cu_all.mat.npy')
cldamt_st_all = np.load('cldamt_st_all.mat.npy')
cldamt_dc_all = np.load('cldamt_dc_all.mat.npy')

crh_q1_all = np.load('crh_q1_all.mat.npy')
crh_q4_all = np.load('crh_q4_all.mat.npy')

buoy_all = np.load('buoy_all.mat.npy')
LHF_all = np.load('LHF_all.mat.npy')
SHF_all = np.load('SHF_all.mat.npy')
qvflux_all = np.load('qvfluxm_all.mat.npy')
TQL_all = np.load('TQL_all.mat.npy')
TQI_all = np.load('TQI_all.mat.npy')
TQLI_all = TQL_all + TQI_all; del TQL_all, TQI_all

In [ ]:
def center_case(time_event,lon_event,lat_event,cri_r=300,ratio=0.5):
    'precipitation area is enclosed within a circle R at reference time'
    
    file_path = TRMM_dir+'2014_2018'
    time_cent = time_event
    lat_cent = lat_event
    lon_cent = lon_event
    
    # read preciptation data in the given domain
    data = subset_TRMM(file_path,time_cent,lat_cent,lon_cent,days=0,lat_inc=5,lon_inc=5)
    prec_sm = data.precipitation.squeeze()
    prec_sum = np.nansum(prec_sm)

    # calculate the distance matrix
    x,y = np.meshgrid(prec_sm.nlon,prec_sm.nlat)
    dist = np.sqrt((x-lon_cent)**2+(y-lat_cent)**2)*100 # distance from center [km]
    prec_sm.values[dist > cri_r] = 0
    prec_frac = (np.nansum(prec_sm)/prec_sum)
    
    if prec_frac >= ratio:
        result = 1
    else:
        result = 0

    return result, prec_frac

In [ ]:
def remove_diurnal_1D(data_all):
    'remove diurnal cycle in the time series [N,time], N cases'
    tmp_re = np.copy(data_all)
    NT = data_all.shape[1]
    
    for n in range(data_all.shape[0]):
        tmp = np.fft.fft(data_all[n,:]) # 49
        freqs = np.array(range(NT))   # This gets the left end correct
        reversed_freqs = freqs[::-1]  # Reverse the array 
        reversed_freqs[0:int(NT/2)] = freqs[1:int(NT/2)+1] # Fix the low end of the reversed array
        freqs = reversed_freqs[::-1]  # reverse it back 
        tmp[np.where(freqs==6)] = 0 # remove diurnal 6 cycles for 6 days
        tmp[np.where(freqs==12)] = 0 # remove half-day
        tmp_re[n,:] = np.fft.ifft(tmp) # reconstructed time-series                   
    
    return tmp_re

In [ ]:
def remove_diurnal_2D(data_all):
    'remove diurnal cycle in the time series [N,time,lev], N cases'
    
    tmp_re = np.copy(data_all)
    NT = data_all.shape[1]
    
    for n in range(data_all.shape[0]):
        for k in range(data_all.shape[2]):
            tmp = np.fft.fft(data_all[n,:,k]) # 49
            freqs = np.array(range(NT))   # This gets the left end correct
            reversed_freqs = freqs[::-1]  # Reverse the array 
            reversed_freqs[0:int(NT/2)] = freqs[1:int(NT/2)+1] # Fix the low end of the reversed array
            freqs = reversed_freqs[::-1]  # reverse it back 
            tmp[np.where(freqs==6)] = 0 # remove diurnal 6 cycles for 6 days
            tmp[np.where(freqs==12)] = 0 # remove half-day
            tmp_re[n,:,k] = np.fft.ifft(tmp) # reconstructed time-series                   
    
    return tmp_re

#### overlapping events identification and removal

In [ ]:
lat_cen = np.arange(-5,10,5)
lon_cen = np.arange(-175,180,5)
idx_record = [] # idx list for final events

for lat in lat_cen:
    for lon in lon_cen:
        idx = np.where(lat_event==lat)[0]
        idy = np.where(lon_event==lon)[0]
        idx_org = np.intersect1d(idx,idy)
        
        time_frame = time_event[idx_org]
        for i,idx in enumerate(idx_org):
            if i == 0:
                if (time_frame[i+1]-time_frame[i]).days >= 6:
                    idx_record.append(idx)
            elif i == len(idx_org)-1:
                if (time_frame[i]-time_frame[i-1]).days >= 6:
                    idx_record.append(idx)
            else:
                if (time_frame[i+1]-time_frame[i]).days >= 6 and \
                   (time_frame[i]-time_frame[i-1]).days >= 6:
                    idx_record.append(idx)            

In [ ]:
idx_rm = np.array(idx_record) # new idx with removal overlapping

In [ ]:
time_event = time_event[idx_rm]
lat_event = lat_event[idx_rm]
lon_event = lon_event[idx_rm]

T_all = T_all[idx_rm,:,:]
q_all = q_all[idx_rm,:,:]
u_all = u_all[idx_rm,:,:]
v_all = v_all[idx_rm,:,:]
N_all = N_all[idx_rm,:]
SCAIbt_all = SCAIbt_all[idx_rm,:]
bt_all = bt_all[idx_rm,:]
llws_all = llws_all[idx_rm,:]
prec_all = prec_all[idx_rm,:]
precstd_all = precstd_all[idx_rm,:]
cwv_all = cwv_all[idx_rm,:]
crh_all = crh_all[idx_rm,:]

omega_all = omega_all[idx_rm,:,:]
rh_all = rh_all[idx_rm,:,:]
cwvstd_all = cwvstd_all[idx_rm,:]
cldfrac_all = cldfrac_all[idx_rm,:]
CFv_all = CFv_all[idx_rm,:,:]
div_all  = div_all[idx_rm,:,:]
cwvten_dyn_all = cwvten_dyn_all[idx_rm,:]
cwvten_phy_all = cwvten_phy_all[idx_rm,:]
cwvten_ana_all = cwvten_ana_all[idx_rm,:]
cwvten_tot_all = cwvten_dyn_all+cwvten_phy_all+cwvten_ana_all
Abar_all = Abar_all[idx_rm,:]
cape_all = cape_all[idx_rm,:]
cin_all = cin_all[idx_rm,:]

cldamt_cu_all = cldamt_cu_all[idx_rm,:]
cldamt_st_all = cldamt_st_all[idx_rm,:]
cldamt_dc_all = cldamt_dc_all[idx_rm,:]

crh_q1_all = crh_q1_all[idx_rm,:]
crh_q4_all = crh_q4_all[idx_rm,:]

buoy_all = buoy_all[idx_rm,:,:]
qvflux_all = qvflux_all[idx_rm,:,:]
SHF_all = SHF_all[idx_rm,:]
LHF_all = LHF_all[idx_rm,:]
TQLI_all = TQLI_all[idx_rm,:]

In [ ]:
# %%time
# # center event
# case_mask = []
# for t in range(10):
#     case_mask.append(center_case(time_event[t],lat_event[t],lon_event[t]
#                                  ,cri_r=300,ratio=0.5))

In [ ]:
# remove diurnal cycle and marginal cases
T_all = remove_diurnal_2D(T_all) 
q_all = remove_diurnal_2D(q_all)
u_all = remove_diurnal_2D(u_all)
v_all = remove_diurnal_2D(v_all)
N_all = N_all
SCAIbt_all = SCAIbt_all
llws_all = remove_diurnal_1D(llws_all)
bt_all = remove_diurnal_1D(bt_all)
prec_all = remove_diurnal_1D(prec_all)
precstd_all = remove_diurnal_1D(prec_all)
cwv_all = remove_diurnal_1D(cwv_all)
crh_all = remove_diurnal_1D(crh_all)

omega_all = remove_diurnal_2D(omega_all)
rh_all = remove_diurnal_2D(rh_all)
cwvstd_all = remove_diurnal_1D(cwvstd_all)
SF_all = remove_diurnal_1D(SF_all)
cldfrac_all = cldfrac_all
CFv_all = remove_diurnal_2D(CFv_all)
div_all  = remove_diurnal_2D(div_all); 
cwvten_dyn_all = remove_diurnal_1D(cwvten_dyn_all)
cwvten_phy_all = remove_diurnal_1D(cwvten_phy_all)
cwvten_ana_all = remove_diurnal_1D(cwvten_ana_all)
cwvten_tot_all = cwvten_dyn_all+cwvten_phy_all+cwvten_ana_all;
Abar_all = Abar_all
cape_all = remove_diurnal_1D(cape_all)
cin_all = remove_diurnal_1D(cin_all)

crh_q1_all = remove_diurnal_1D(crh_q1_all)
crh_q4_all = remove_diurnal_1D(crh_q4_all)

cldamt_cu_all = remove_diurnal_1D(cldamt_cu_all)
cldamt_st_all = remove_diurnal_1D(cldamt_st_all)
cldamt_dc_all = remove_diurnal_1D(cldamt_dc_all)

SHF_all = remove_diurnal_1D(SHF_all)
LHF_all = remove_diurnal_1D(LHF_all)
SFX_all = SHF_all + LHF_all
TQLI_all = remove_diurnal_1D(TQLI_all)

#SFXstd_all = remove_diurnal_1D(SFXstd_all)
qvflux_all = remove_diurnal_2D(qvflux_all)

In [ ]:
def idx_season(time_event):
    "return idx for four seasons"
    idx_DJF = [];idx_MAM = []
    idx_JJA = [];idx_SON = []
    for i,time in enumerate(time_event):
        if time.month == 12 or time.month == 1 or time.month == 2:
            idx_DJF.append(i)
        if time.month == 3 or time.month == 4 or time.month == 5:
            idx_MAM.append(i)
        if time.month == 6 or time.month == 7 or time.month == 8:
            idx_JJA.append(i)
        if time.month == 9 or time.month == 10 or time.month == 11:
            idx_SON.append(i)
    
    return idx_DJF,idx_MAM,idx_JJA,idx_SON

#### TRMM precipitation seasonality showcases

In [ ]:
#os.chdir(TRMM_dir+'2014_2018')

In [ ]:
# date = []; date_begin = datetime(2014,1,1)
# for t in range(14608):
#     tmp = date_begin + t*timedelta(hours=3)
#     date.append(tmp)
    
# # TRMM3B42 dataset 2014, 3hrly separate
# yr = ['2014_2018']
# prec_3hr = np.nan*np.zeros((len(date),241,1440))

# n=1
# files = sorted(glob('3B42.*'))
# for file in files:
#     data = xr.open_dataset(file).precipitation        
#     prec_3hr[n-1,:,:] = data.values.swapaxes(0,1)
#     n+=1
# lon = data.nlon; lat = data.nlat
# prec_xr = xr.DataArray(prec_3hr,coords=[date,lat,lon],dims=['time','lat','lon']); del prec_3hr

In [ ]:
# prec = prec_xr.sel(lat=slice(-20,20)) # indian ocean
# prec_clim = prec.groupby('time.month').mean('time').compute()

In [ ]:
# fig = plt.figure(figsize=(13,4))
# ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))

# clevs=np.linspace(0,20,21)
# ps = plt.contourf(prec_clim.lon,prec_clim.lat,prec_clim.mean(axis=0)*24,transform=ccrs.PlateCarree(),
#                   levels=clevs,cmap='jet',)
# cbar = plt.colorbar(fraction=0.05,pad=0.1,orientation='horizontal')
# cbar.set_label('[mm/day]',fontsize=12)
# #cs = plt.contour(prec_clim.lon,prec_clim.lat,prec_clim.mean(axis=0)*24)
# ax.coastlines()
# ax.add_feature(cfeat.LAND,zorder=1,edgecolor='k',facecolor='whitesmoke') # maskout LAND
# #gl = ax.gridlines(draw_labels=False,linestyle='--')
# #gl.xlabels_top = gl.ylabels_right = False
# #gl.xformatter = LONGITUDE_FORMATTER
# #gl.yformatter = LATITUDE_FORMATTER
# ax.set_yticks([-20,-15,-10,-5,0,5,10,15,20])
# ax.set_yticklabels(['20S','15S','10S','5S','EQ','5N','10N','15N','20N'])
# ax.set_xticks([-179.875,-120,-60,-0,60,120,179.875])
# ax.set_xticklabels(['0','60E','120E','180','120W','60W','0'])
# ax.set_ylim([-20,20]);ax.grid(linestyle=':')
# plt.title('TRMM 3B42 precipitation, 2014-2018, annual mean',fontsize=14)

In [ ]:
# os.chdir('/w2-data/willytsai/ISCCP/fig/')
# fig.savefig('TRMMprecip_regimes_demo.pdf',dpi=200)

In [ ]:
# # explanation rate 
# lon_bin = np.arange(-175,185,5)
# lat_bin = np.arange(-5,10,5)

# precsum_total = np.zeros((len(lon_bin),len(lat_bin)))
# for i,lon_cen in enumerate(lon_bin):
#     for j,lat_cen in enumerate(lat_bin):
#         precsum_total[i,j] = prec_xr.sel(lat=slice(lat_cen-5,lat_cen+5),lon=slice(lon_cen-5,lon_cen+5)).sum()

In [ ]:
# precsum_event = np.copy(precsum_total)*0
# for n in range(len(time_event)):
    
#     for i,lon_cen in enumerate(lon_bin):
#         for j,lat_cen in enumerate(lat_bin):
#             if lon_cen == lon_event[n] and lat_cen == lat_event[n]:
#                 precsum_event[i,j] += prec_all[n,:].sum()*1600 

In [ ]:
# # convert lon_bin, lat_bin for pcolormap
# lat_edge = np.arange(-7.5,12.5,5)
# lon_edge = np.arange(-182.5,182.5,5)

# cover_ratio = precsum_event/precsum_total # explained total rainfall

# cover_ratio = np.vstack([np.zeros((cover_ratio.shape[1])),cover_ratio])
# cover_ratio = np.vstack([cover_ratio.T,np.zeros((cover_ratio.shape[0])).T]).T
# cover_ratio[cover_ratio==0] = np.nan

In [ ]:
# fig = plt.figure(figsize=(13,4))
# ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))

# clevs=np.linspace(0,20,21)
# ps = plt.pcolormesh(lon_edge,lat_edge,cover_ratio.T,transform=ccrs.PlateCarree(),
#                   cmap='jet',)
# cbar = plt.colorbar(fraction=0.05,pad=0.1,orientation='horizontal')
# cbar.set_label('[mm/day]',fontsize=12)
# #cs = plt.contour(prec_clim.lon,prec_clim.lat,prec_clim.mean(axis=0)*24)
# ax.coastlines()
# ax.add_feature(cfeat.LAND,zorder=1,edgecolor='k',facecolor='whitesmoke') # maskout LAND
# #gl = ax.gridlines(draw_labels=False,linestyle='--')
# #gl.xlabels_top = gl.ylabels_right = False
# #gl.xformatter = LONGITUDE_FORMATTER
# #gl.yformatter = LATITUDE_FORMATTER

# ax.set_yticks([-20,-15,-10,-5,0,5,10,15,20])
# ax.set_yticklabels(['20S','15S','10S','5S','EQ','5N','10N','15N','20N'])
# ax.set_xticks([-179.875,-120,-60,-0,60,120,179.875])
# ax.set_xticklabels(['0','60E','120E','180','120W','60W','0'])
# ax.set_ylim([-20,20]);ax.grid(linestyle=':')

#### Region definition

In [ ]:
idx_IND = event_region(lat_event,lon_event,60,90,-10,10) # Tropical Indian Ocean
idx_WPC = event_region(lat_event,lon_event,150,180,-10,10) # Tropical Western Pacific
idx_EPC = event_region(lat_event,lon_event,-180,-90,-10,10) # Tropical Eastern Pacific

In [ ]:
len(idx_IND)+len(idx_WPC)+len(idx_EPC)

In [ ]:
cldfrac_IND = cldfrac_all[idx_IND,:] # reference cloud coverage 
cldfrac_WPC = cldfrac_all[idx_WPC,:] # reference cloud coverage 
cldfrac_EPC = cldfrac_all[idx_EPC,:] # reference cloud coverage 

prec_IND = prec_all[idx_IND,:] 
prec_WPC = prec_all[idx_WPC,:] 
prec_EPC = prec_all[idx_EPC,:] 

crh_IND = crh_all[idx_IND,:] 
crh_WPC = crh_all[idx_WPC,:] 
crh_EPC = crh_all[idx_EPC,:] 

Abar_IND = Abar_all[idx_IND,:]
Abar_WPC = Abar_all[idx_WPC,:]
Abar_EPC = Abar_all[idx_EPC,:]

SCAIbt_IND = SCAIbt_all[idx_IND,:]
SCAIbt_WPC = SCAIbt_all[idx_WPC,:]
SCAIbt_EPC = SCAIbt_all[idx_EPC,:]

N_IND = N_all[idx_IND,:]
N_WPC = N_all[idx_WPC,:]
N_EPC = N_all[idx_EPC,:]

q_IND = q_all[idx_IND,:,:]
q_WPC = q_all[idx_WPC,:,:]
q_EPC = q_all[idx_EPC,:,:]

buoy_IND = buoy_all[idx_IND,:,:]
buoy_WPC = buoy_all[idx_WPC,:,:]
buoy_EPC = buoy_all[idx_EPC,:,:]

rh_IND = rh_all[idx_IND,:,:]
rh_WPC = rh_all[idx_WPC,:,:]
rh_EPC = rh_all[idx_EPC,:,:]

#------------------------------------------------
counts_IND,bins = np.histogram(cldfrac_IND[:,24],bins=10,range=(0,1))
plt.plot(bins[1:],counts_IND/sum(counts_IND),'-r',markerfacecolor='w')
counts_WPC,bins = np.histogram(cldfrac_WPC[:,24],bins=10,range=(0,1))
plt.plot(bins[1:],counts_WPC/sum(counts_WPC),'-s',color='orange',markerfacecolor='w')
counts_EPC,bins = np.histogram(cldfrac_EPC[:,24],bins=10,range=(0,1))
plt.plot(bins[1:],counts_EPC/sum(counts_EPC),'-sb',markerfacecolor='w')
plt.ylabel('Probability',fontsize=12);plt.xlabel('Event cloud cover',fontsize=12)
legend=plt.legend(['IND '+'('+str(sum(counts_IND))+')',
            'WPC '+'('+str(sum(counts_WPC))+')',
            'EPC '+'('+str(sum(counts_EPC))+')'])
plt.setp(legend.get_texts(),color='dimgrey')
plt.xlim([-0.05,1.05]);plt.grid(linestyle=':');plt.show()

counts_IND,bins = np.histogram(Abar_IND[:,24],bins=10,range=(0,400))
plt.plot(bins[1:],counts_IND/sum(counts_IND),'-sr',markerfacecolor='w')
counts_WPC,bins = np.histogram(Abar_WPC[:,24],bins=10,range=(0,400))
plt.plot(bins[1:],counts_WPC/sum(counts_WPC),'-s',color='orange',markerfacecolor='w')
counts_EPC,bins = np.histogram(Abar_EPC[:,24],bins=10,range=(0,400))
plt.plot(bins[1:],counts_EPC/sum(counts_EPC),'-sb',markerfacecolor='w')
plt.ylabel('Probability',fontsize=12);plt.xlabel('Cloud coverage',fontsize=12)
legend=plt.legend(['IND '+'('+str(sum(counts_IND))+')',
            'WPC '+'('+str(sum(counts_WPC))+')',
            'EPC '+'('+str(sum(counts_EPC))+')'])
plt.setp(legend.get_texts(),color='dimgrey')
plt.xlim([0,400]);plt.grid(linestyle=':');plt.show()

In [ ]:
# CRH vs cloud cover for all regimes
crh_cri = np.array([0,0.5,0.65,0.8,1])
cover_cri = np.array([0,0.2,0.4,0.6,0.8,1])
cover_bin = np.zeros((len(crh_cri)-1,len(cover_cri)-1))
precm_bin = np.copy(cover_bin)
precste_bin = np.copy(cover_bin)
SCAIbt_bin = np.copy(cover_bin)
SCAIbtste_bin = np.copy(cover_bin)

for n in range(len(crh_cri)-1):
    idx = np.where(np.logical_and(crh_all[:,24]>=crh_cri[n],crh_all[:,24]<crh_cri[n+1]))[0]
    
    cover_tmp = cldfrac_all[idx,:]
    prec_tmp = prec_all[idx,:]
    Abar_tmp = Abar_all[idx,:]
    SCAIbt_tmp = SCAIbt_all[idx,:]
    
    for i in range(len(cover_cri)-1):
        idx2 = np.where(np.logical_and(cover_tmp[:,24]>=cover_cri[i]
                                      ,cover_tmp[:,24]<cover_cri[i+1]))[0]
        cover_bin[n,i] = len(idx2)
        precm_bin[n,i] = np.nanmean(prec_tmp[idx2,24])
        precste_bin[n,i] = np.nanstd(prec_tmp[idx2,24])/(len(idx2)**0.5)
        SCAIbt_bin[n,i] = np.nanmean(SCAIbt_tmp[idx2,24])
        SCAIbtste_bin[n,i] = np.nanstd(SCAIbt_tmp[idx2,24])/(len(idx2)**0.5)
        

In [ ]:
fig, ax = plt.subplots(1,4,figsize=(14,3.5))

ax[0].bar(['0','0.2','0.4','0.6','0.8'],cover_bin[0,:]/sum(cover_bin[0,:]),width=0.6,alpha=0.5)
ax[0].set_xticks([-0.5,0.5,1.5,2.5,3.5]);ax[0].set_xticklabels(['0','0.2','0.4','0.6','0.8'])
ax[0].set_xlabel('Cloud cover',fontsize=12);ax[0].set_ylabel('Probability',fontsize=12)
ax[0].set_title('CRH_ref: 0-0.5, All',fontsize=12)
ax[0].grid(axis='y',linestyle=':');ax[0].text(x=3.5,y=0.90,s=str(int(sum(cover_bin[0,:]))),color='grey')
ax[0].set_ylim([0,1])

ax[1].bar(['0','0.2','0.4','0.6','0.8'],cover_bin[1,:]/sum(cover_bin[1,:]),width=0.6,alpha=0.5)
ax[1].set_xticks([-0.5,0.5,1.5,2.5,3.5]);ax[1].set_xticklabels(['0','0.2','0.4','0.6','0.8'])
ax[1].set_xlabel('Cloud cover',fontsize=12);ax[1].set_ylabel('Probability',fontsize=12)
ax[1].set_title('CRH_ref: 0.5-0.65, All',fontsize=12)
ax[1].grid(axis='y',linestyle=':');ax[1].text(x=3.5,y=0.63,s=str(int(sum(cover_bin[1,:]))),color='grey')
ax[1].set_ylim([0,0.7])

ax[2].bar(['0','0.2','0.4','0.6','0.8'],cover_bin[2,:]/sum(cover_bin[2,:]),width=0.6,alpha=0.5)
ax[2].set_xticks([-0.5,0.5,1.5,2.5,3.5]);ax[2].set_xticklabels(['0','0.2','0.4','0.6','0.8'])
ax[2].set_xlabel('Cloud cover',fontsize=12);ax[2].set_ylabel('Probability',fontsize=12)
ax[2].set_title('CRH_ref: 0.65-0.8, All',fontsize=12)
ax[2].grid(axis='y',linestyle=':');ax[2].text(x=3.5,y=0.63,s=str(int(sum(cover_bin[2,:]))),color='grey')
ax[2].set_ylim([0,0.7])

ax[3].bar(['0','0.2','0.4','0.6','0.8'],cover_bin[3,:]/sum(cover_bin[3,:]),width=0.6,alpha=0.5)
ax[3].set_xticks([-0.5,0.5,1.5,2.5,3.5]);ax[3].set_xticklabels(['0','0.2','0.4','0.6','0.8'])
ax[3].set_xlabel('Cloud cover',fontsize=12);ax[3].set_ylabel('Probability',fontsize=12)
ax[3].set_title('CRH_ref: 0.8-1, All',fontsize=12)
ax[3].grid(axis='y',linestyle=':');ax[3].text(x=3.5,y=0.63,s=str(int(sum(cover_bin[3,:]))),color='grey')
ax[3].set_ylim([0,0.7])

# adding precipitation_mean variations in each subplot
ax0 = ax[0].twinx()
#ax0.plot([0,1,2,3,4],precm_bin[0,:]*24,'xr');ax0.set_ylim([5,80])
ax0.errorbar([0,1,2,3,4],SCAIbt_bin[0,:],yerr=SCAIbtste_bin[0,:],fmt='or');ax0.set_ylim([0,15])
ax0.tick_params(axis='y', labelcolor='r');ax0.set_ylabel('SCAI',color='r')

ax1 = ax[1].twinx()
#ax1.plot([0,1,2,3,4],precm_bin[1,:]*24,'xr');ax1.set_ylim([5,80])
ax1.errorbar([0,1,2,3,4],SCAIbt_bin[1,:],yerr=SCAIbtste_bin[1,:],fmt='or');ax1.set_ylim([0,15])
ax1.tick_params(axis='y', labelcolor='r');ax1.set_ylabel('SCAI',color='r')

ax2 = ax[2].twinx()
#ax2.plot([0,1,2,3,4],precm_bin[2,:]*24,'xr');ax2.set_ylim([5,80])
ax2.errorbar([0,1,2,3,4],SCAIbt_bin[2,:],yerr=SCAIbtste_bin[2,:],fmt='or');ax2.set_ylim([0,15])
ax2.tick_params(axis='y', labelcolor='r');ax2.set_ylabel('SCAI',color='r')

ax3 = ax[3].twinx()
#ax3.plot([0,1,2,3,4],precm_bin[3,:]*24,'xr');ax3.set_ylim([5,80])
ax3.errorbar([0,1,2,3,4],SCAIbt_bin[3,:],yerr=SCAIbtste_bin[3,:],fmt='or');ax3.set_ylim([0,15])
ax3.tick_params(axis='y', labelcolor='r');ax3.set_ylabel('SCAI',color='r')

plt.tight_layout()

fig.savefig(fig_dir+'CRH_CF_groups_SCAI_All.pdf',dip=200,bbox_inchs='tight')

#### Stats for reference time

In [ ]:
# cloud cover & Abar
#cover_cri = np.array([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])
cover_cri = np.linspace(0,1,21)
Abar_bin = np.zeros((3,len(cover_cri)-1))
Abarstd_bin = np.copy(Abar_bin)

for i in range(len(cover_cri)-1):
    idx1 = np.where(np.logical_and(cldfrac_IND[:,24]>=cover_cri[i]
                                 ,cldfrac_IND[:,24]<cover_cri[i+1]))[0]
    idx2 = np.where(np.logical_and(cldfrac_WPC[:,24]>=cover_cri[i]
                                 ,cldfrac_WPC[:,24]<cover_cri[i+1]))[0]
    idx3 = np.where(np.logical_and(cldfrac_EPC[:,24]>=cover_cri[i]
                                 ,cldfrac_EPC[:,24]<cover_cri[i+1]))[0]
    Abar_tmp1 = Abar_IND[idx1,24]
    Abar_tmp1 = Abar_tmp1[~np.isnan(Abar_tmp1)]
    Abar_tmp2 = Abar_WPC[idx2,24]
    Abar_tmp2 = Abar_tmp2[~np.isnan(Abar_tmp2)]    
    Abar_tmp3 = Abar_EPC[idx3,24]
    Abar_tmp3 = Abar_tmp3[~np.isnan(Abar_tmp3)]
    
    Abar_bin[0,i] = np.nanmean(Abar_tmp1)
    Abar_bin[1,i] = np.nanmean(Abar_tmp2)
    Abar_bin[2,i] = np.nanmean(Abar_tmp3)
    
    Abarstd_bin[0,i] = np.nanstd(Abar_tmp1)/len(idx1)**0.5
    Abarstd_bin[1,i] = np.nanstd(Abar_tmp2)/len(idx2)**0.5
    Abarstd_bin[2,i] = np.nanstd(Abar_tmp3)/len(idx3)**0.5
    
    #Abarstd_bin[i] = np.nanstd(Abar_all[idx,24])/len(idx)**0.5

In [ ]:
# cloud cover & N
#cover_cri = np.array([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])
cover_cri = np.linspace(0,1,21)
N_bin = np.zeros((3,len(cover_cri)-1))
Nstd_bin = np.copy(N_bin)

for i in range(len(cover_cri)-1):
    idx1 = np.where(np.logical_and(cldfrac_IND[:,24]>=cover_cri[i]
                                 ,cldfrac_IND[:,24]<cover_cri[i+1]))[0]
    idx2 = np.where(np.logical_and(cldfrac_WPC[:,24]>=cover_cri[i]
                                 ,cldfrac_WPC[:,24]<cover_cri[i+1]))[0]
    idx3 = np.where(np.logical_and(cldfrac_EPC[:,24]>=cover_cri[i]
                                 ,cldfrac_EPC[:,24]<cover_cri[i+1]))[0]
    N_tmp1 = N_IND[idx1,24]
    N_tmp1 = N_tmp1[~np.isnan(N_tmp1)]
    N_tmp2 = N_WPC[idx2,24]
    N_tmp2 = N_tmp2[~np.isnan(N_tmp2)]    
    N_tmp3 = N_EPC[idx3,24]
    N_tmp3 = N_tmp3[~np.isnan(N_tmp3)]
    
    N_bin[0,i] = np.nanmean(N_tmp1)
    N_bin[1,i] = np.nanmean(N_tmp2)
    N_bin[2,i] = np.nanmean(N_tmp3)
    
    Nstd_bin[0,i] = np.nanstd(N_tmp1)/len(idx1)**0.5
    Nstd_bin[1,i] = np.nanstd(N_tmp2)/len(idx2)**0.5
    Nstd_bin[2,i] = np.nanstd(N_tmp3)/len(idx3)**0.5
    
    #Abarstd_bin[i] = np.nanstd(Abar_all[idx,24])/len(idx)**0.5

In [ ]:
fig = plt.figure(figsize=(5,5))

plt.plot(np.linspace(0.05,0.95,20),Abar_bin[0,:],linewidth=2,color='b')
plt.fill_between(np.linspace(0.05,0.95,20),Abar_bin[0,:]+Abarstd_bin[0,:],Abar_bin[0,:],
                color='b',alpha=0.2)
plt.fill_between(np.linspace(0.05,0.95,20),Abar_bin[0,:]-Abarstd_bin[0,:],Abar_bin[0,:],
                color='b',alpha=0.2)

plt.plot(np.linspace(0.05,0.95,20),Abar_bin[1,:],linewidth=2,color='g')
plt.fill_between(np.linspace(0.05,0.95,20),Abar_bin[1,:]+Abarstd_bin[1,:],Abar_bin[1,:],
                color='g',alpha=0.2)
plt.fill_between(np.linspace(0.05,0.95,20),Abar_bin[1,:]-Abarstd_bin[1,:],Abar_bin[1,:],
                color='g',alpha=0.2)

plt.plot(np.linspace(0.05,0.95,20),Abar_bin[2,:],linewidth=2,color='darkorange')
plt.fill_between(np.linspace(0.05,0.95,20),Abar_bin[2,:]+Abarstd_bin[2,:],Abar_bin[2,:],
                color='darkorange',alpha=0.2)
plt.fill_between(np.linspace(0.05,0.95,20),Abar_bin[2,:]-Abarstd_bin[2,:],Abar_bin[2,:],
                color='darkorange',alpha=0.2)
plt.xlim([0,0.9])
plt.ylabel(r'$\sqrt{\bar{A}}$   [km]',fontsize=12);plt.xlabel('Cloud coverage',fontsize=12)
legend=plt.legend(['IND '+'('+str(sum(counts_IND))+')',
            'WPC '+'('+str(sum(counts_WPC))+')',
            'EPC '+'('+str(sum(counts_EPC))+')'],loc=2)
plt.setp(legend.get_texts(),color='dimgrey')
plt.grid(linestyle=':');plt.show()

In [ ]:
fig = plt.figure(figsize=(5,5))

plt.plot(np.linspace(0.05,0.95,20),N_bin[0,:],linewidth=2,color='b')
plt.fill_between(np.linspace(0.05,0.95,20),N_bin[0,:]+Nstd_bin[0,:],N_bin[0,:],
                color='b',alpha=0.2)
plt.fill_between(np.linspace(0.05,0.95,20),N_bin[0,:]-Nstd_bin[0,:],N_bin[0,:],
                color='b',alpha=0.2)

plt.plot(np.linspace(0.05,0.95,20),N_bin[1,:],linewidth=2,color='g')
plt.fill_between(np.linspace(0.05,0.95,20),N_bin[1,:]+Nstd_bin[1,:],N_bin[1,:],
                color='g',alpha=0.2)
plt.fill_between(np.linspace(0.05,0.95,20),N_bin[1,:]-Nstd_bin[1,:],N_bin[1,:],
                color='g',alpha=0.2)

plt.plot(np.linspace(0.05,0.95,20),N_bin[2,:],linewidth=2,color='darkorange')
plt.fill_between(np.linspace(0.05,0.95,20),N_bin[2,:]+Nstd_bin[2,:],N_bin[2,:],
                color='darkorange',alpha=0.2)
plt.fill_between(np.linspace(0.05,0.95,20),N_bin[2,:]-Nstd_bin[2,:],N_bin[2,:],
                color='darkorange',alpha=0.2)
plt.xlim([0,0.9])
plt.ylabel('Number',fontsize=12);plt.xlabel('Cloud coverage',fontsize=12)
#plt.legend(['IND','WPC','EPC'],fontsize=12,loc=2)
plt.grid(linestyle=':');plt.show()

In [ ]:
# qv, rh profile binned by cloud coverage
cover_cri = np.linspace(0,0.8,5)
qv_binIND = np.zeros((len(cover_cri)-1,len(lev)))
qvstd_binIND = np.copy(qv_binIND)
qv_binWPC = np.zeros((len(cover_cri)-1,len(lev)))
qvstd_binWPC = np.copy(qv_binIND)
qv_binEPC = np.zeros((len(cover_cri)-1,len(lev)))
qvstd_binEPC = np.copy(qv_binIND)

for i in range(len(cover_cri)-1):
    idx1 = np.where(np.logical_and(cldfrac_IND[:,24]>=cover_cri[i]
                                 ,cldfrac_IND[:,24]<cover_cri[i+1]))[0]
    idx2 = np.where(np.logical_and(cldfrac_WPC[:,24]>=cover_cri[i]
                                 ,cldfrac_WPC[:,24]<cover_cri[i+1]))[0]
    idx3 = np.where(np.logical_and(cldfrac_EPC[:,24]>=cover_cri[i]
                                 ,cldfrac_EPC[:,24]<cover_cri[i+1]))[0]
    q_tmp1 = q_IND[idx1,24,:]
    q_tmp2 = q_WPC[idx2,24,:]
    q_tmp3 = q_EPC[idx3,24,:]
    
    qv_binIND[i,:] = np.nanmean(q_tmp1,axis=0)
    qv_binWPC[i,:] = np.nanmean(q_tmp2,axis=0)
    qv_binEPC[i,:] = np.nanmean(q_tmp3,axis=0)

fig,ax = plt.subplots(1,3,figsize=(12,4))
colors = ['r','g','b','k']
for n in range(4):
    ax[0].plot(qv_binIND[n,:]*1000,lev,color=colors[n])
    ax[1].plot(qv_binWPC[n,:]*1000,lev,color=colors[n])
    ax[2].plot(qv_binEPC[n,:]*1000,lev,color=colors[n])
ax[0].set_ylim([1000,100]);ax[0].set_xlim([-0.5,20])
ax[1].set_ylim([1000,100]);ax[1].set_xlim([-0.5,20])
ax[2].set_ylim([1000,100]);ax[2].set_xlim([-0.5,20])
ax[0].text(1,200,'IND',fontsize=16,color='grey')
ax[1].text(1,200,'WPC',fontsize=16,color='grey')
ax[2].text(1,200,'EPC',fontsize=16,color='grey')
ax[0].legend(['0-0.2','0.2-0.4','0.4-0.6','0.6-0..8'])
ax[0].grid(linestyle=':');ax[1].grid(linestyle=':');ax[2].grid(linestyle=':')
ax[0].set_ylabel('Pressure [hPa]',fontsize=12)
ax[0].set_xlabel('qv [g/kg]',fontsize=12)
ax[1].set_xlabel('qv [g/kg]',fontsize=12)
ax[2].set_xlabel('qv [g/kg]',fontsize=12)

plt.tight_layout()

In [ ]:
cover_cri = np.linspace(0,0.8,5)
rh_binIND = np.zeros((len(cover_cri)-1,len(lev_m)))
rh_binWPC = np.zeros((len(cover_cri)-1,len(lev_m)))
rh_binEPC = np.zeros((len(cover_cri)-1,len(lev_m)))

for i in range(len(cover_cri)-1):
    idx1 = np.where(np.logical_and(cldfrac_IND[:,24]>=cover_cri[i]
                                 ,cldfrac_IND[:,24]<cover_cri[i+1]))[0]
    idx2 = np.where(np.logical_and(cldfrac_WPC[:,24]>=cover_cri[i]
                                 ,cldfrac_WPC[:,24]<cover_cri[i+1]))[0]
    idx3 = np.where(np.logical_and(cldfrac_EPC[:,24]>=cover_cri[i]
                                 ,cldfrac_EPC[:,24]<cover_cri[i+1]))[0]
    rh_tmp1 = rh_IND[idx1,24,14:] # to 1 hpa
    rh_tmp2 = rh_WPC[idx2,24,14:]
    rh_tmp3 = rh_EPC[idx3,24,14:]
    
    rh_binIND[i,:] = np.nanmean(rh_tmp1,axis=0)
    rh_binWPC[i,:] = np.nanmean(rh_tmp2,axis=0)
    rh_binEPC[i,:] = np.nanmean(rh_tmp3,axis=0)

fig,ax = plt.subplots(1,3,figsize=(12,4))
colors = ['r','g','b','k']
for n in range(4):
    ax[0].plot(rh_binIND[n,:]*100,np.flipud(lev_m),color=colors[n])
    ax[1].plot(rh_binWPC[n,:]*100,np.flipud(lev_m),color=colors[n])
    ax[2].plot(rh_binEPC[n,:]*100,np.flipud(lev_m),color=colors[n])
ax[0].set_ylim([1000,100]);ax[0].set_xlim([30,100])
ax[1].set_ylim([1000,100]);ax[1].set_xlim([30,100])
ax[2].set_ylim([1000,100]);ax[2].set_xlim([30,100])
ax[0].text(35,200,'IND',fontsize=16,color='grey')
ax[1].text(35,200,'WPC',fontsize=16,color='grey')
ax[2].text(35,200,'EPC',fontsize=16,color='grey')
ax[0].legend(['0-0.2','0.2-0.4','0.4-0.6','0.6-0..8'],loc=3)
ax[0].grid(linestyle=':');ax[1].grid(linestyle=':');ax[2].grid(linestyle=':')
ax[0].set_ylabel('Pressure [hPa]',fontsize=12)
ax[0].set_xlabel('RH [%]',fontsize=12)
ax[1].set_xlabel('RH [%]',fontsize=12)
ax[2].set_xlabel('RH [%]',fontsize=12)

plt.tight_layout()

In [ ]:
cover_cri = np.linspace(0,0.8,5)
buoy_binIND = np.zeros((len(cover_cri)-1,len(lev)))
buoy_binWPC = np.zeros((len(cover_cri)-1,len(lev)))
buoy_binEPC = np.zeros((len(cover_cri)-1,len(lev)))

for i in range(len(cover_cri)-1):
    idx1 = np.where(np.logical_and(cldfrac_IND[:,24]>=cover_cri[i]
                                 ,cldfrac_IND[:,24]<cover_cri[i+1]))[0]
    idx2 = np.where(np.logical_and(cldfrac_WPC[:,24]>=cover_cri[i]
                                 ,cldfrac_WPC[:,24]<cover_cri[i+1]))[0]
    idx3 = np.where(np.logical_and(cldfrac_EPC[:,24]>=cover_cri[i]
                                 ,cldfrac_EPC[:,24]<cover_cri[i+1]))[0]
    buoy_tmp1 = buoy_IND[idx1,:,16] # to 1 hpa
    buoy_tmp2 = buoy_WPC[idx2,:,16]
    buoy_tmp3 = buoy_EPC[idx3,:,16]
    
    buoy_binIND[i,:] = np.nanmean(buoy_tmp1,axis=0)
    buoy_binWPC[i,:] = np.nanmean(buoy_tmp2,axis=0)
    buoy_binEPC[i,:] = np.nanmean(buoy_tmp3,axis=0)
    
fig,ax = plt.subplots(1,3,figsize=(12,4))
colors = ['r','g','b','k']
for n in range(4):
    ax[0].plot(buoy_binIND[n,:],lev,color=colors[n])
    ax[1].plot(buoy_binWPC[n,:],lev,color=colors[n])
    ax[2].plot(buoy_binEPC[n,:],lev,color=colors[n])
ax[0].set_ylim([1000,100]);ax[0].set_xlim([-2.5,5])
ax[1].set_ylim([1000,100]);ax[1].set_xlim([-2.5,5])
ax[2].set_ylim([1000,100]);ax[2].set_xlim([-2.5,5])
ax[0].text(-5,200,'IND',fontsize=16,color='grey')
ax[1].text(-5,200,'WPC',fontsize=16,color='grey')
ax[2].text(-5,200,'EPC',fontsize=16,color='grey')
ax[0].legend(['0-0.2','0.2-0.4','0.4-0.6','0.6-0..8'],loc=3)
ax[0].grid(linestyle=':');ax[1].grid(linestyle=':');ax[2].grid(linestyle=':')
ax[0].set_ylabel('Pressure [hPa]',fontsize=12)
ax[0].set_xlabel('RH [%]',fontsize=12)
ax[1].set_xlabel('RH [%]',fontsize=12)
ax[2].set_xlabel('RH [%]',fontsize=12)

plt.tight_layout()

In [ ]:
# CRH vs cloud cover 
crh_cri = np.array([0,0.5,0.65,0.8,1])
cover_cri = np.array([0,0.2,0.4,0.6,0.8,1])
cover_bin = np.zeros((len(crh_cri)-1,len(cover_cri)-1))
precm_bin = np.copy(cover_bin)
precste_bin = np.copy(cover_bin)
SCAIbt_bin = np.copy(cover_bin)
SCAIbtste_bin = np.copy(cover_bin)

for n in range(len(crh_cri)-1):
    idx = np.where(np.logical_and(crh_EPC[:,24]>=crh_cri[n],crh_EPC[:,24]<crh_cri[n+1]))[0]
    
    cover_tmp = cldfrac_EPC[idx,:]
    prec_tmp = prec_EPC[idx,:]
    SCAIbt_tmp = SCAIbt_EPC[idx,:]
    
    for i in range(len(cover_cri)-1):
        idx2 = np.where(np.logical_and(cover_tmp[:,24]>=cover_cri[i]
                                      ,cover_tmp[:,24]<cover_cri[i+1]))[0]
        cover_bin[n,i] = len(idx2)
        precm_bin[n,i] = np.nanmean(prec_tmp[idx2,24])
        precste_bin[n,i] = np.nanstd(prec_tmp[idx2,24])/len(idx2)**0.5
        SCAIbt_bin[n,i] = np.nanmean(SCAIbt_tmp[idx2,24])
        SCAIbtste_bin[n,i] = np.nanstd(SCAIbt_tmp[idx2,24])/len(idx2)**0.5

In [ ]:
# joint histogram CRH, cloud fraction for all 
crh_bin = np.linspace(0,1,21)
cldfrac_bin = np.linspace(0,1,21)
hist_bin_cf = np.zeros((len(crh_bin)-1,len(cldfrac_bin)-1))
hist_bin_cfp = np.zeros((len(crh_bin)-1,len(cldfrac_bin)-1))

for i in range(len(cldfrac_bin)-1):
    for j in range(len(crh_bin)-1):
        
        idx = np.where(np.logical_and(crh_all[:,24]>=crh_bin[i],crh_all[:,24]<crh_bin[i+1]))[0]
        idx2 = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[j]
                                       ,cldfrac_all[:,24]<cldfrac_bin[j+1]))[0]
        idx = np.intersect1d(idx,idx2)
        hist_bin_cf[i,j] = len(idx)
        hist_bin_cfp[i,j] = np.nanmean(prec_all[idx,24])

In [ ]:
# joint histogram CRH, Abar for all 
Abar_bin = np.linspace(0,400,21)
crh_bin = np.linspace(0,1,21)
hist_bin_A = np.zeros((len(Abar_bin)-1,len(cldfrac_bin)-1))
hist_bin_Ap = np.zeros((len(Abar_bin)-1,len(cldfrac_bin)-1))

for i in range(len(Abar_bin)-1):
    for j in range(len(crh_bin)-1):
        
        idx = np.where(np.logical_and(crh_all[:,24]>=crh_bin[i],crh_all[:,24]<crh_bin[i+1]))[0]
        idx2 = np.where(np.logical_and(Abar_all[:,24]>=Abar_bin[j]
                                       ,Abar_all[:,24]<Abar_bin[j+1]))[0]
        idx = np.intersect1d(idx,idx2)
        hist_bin_A[i,j] = len(idx)
        hist_bin_Ap[i,j] = np.nanmean(prec_all[idx,24])

In [ ]:
# add row and column for pcolor
hist_bin_cf = np.vstack([np.zeros((hist_bin_cf.shape[1])),hist_bin_cf])
hist_bin_cf = np.vstack([hist_bin_cf.T,np.zeros((hist_bin_cf.shape[0])).T]).T

hist_bin_cfp = np.vstack([np.zeros((hist_bin_cfp.shape[1])),hist_bin_cfp])
hist_bin_cfp = np.vstack([hist_bin_cfp.T,np.zeros((hist_bin_cfp.shape[0])).T]).T

hist_bin_A = np.vstack([np.zeros((hist_bin_A.shape[1])),hist_bin_A])
hist_bin_A = np.vstack([hist_bin_A.T,np.zeros((hist_bin_A.shape[0])).T]).T

hist_bin_Ap = np.vstack([np.zeros((hist_bin_Ap.shape[1])),hist_bin_Ap])
hist_bin_Ap = np.vstack([hist_bin_Ap.T,np.zeros((hist_bin_Ap.shape[0])).T]).T

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(12,5))
ps1 = ax[0].pcolormesh(crh_bin,cldfrac_bin,np.log10(hist_bin_cf.T/np.sum(hist_bin_cf))[:-1,:-1])
cbar = plt.colorbar(ps1,ax=ax[0],shrink=0.8)
cbar.set_label('Log$_{10}$(Probability)',fontsize=13)
ax[0].set_xlabel('Column relative humidity',fontsize=14);ax[0].set_ylabel('Cloud cover',fontsize=14)
ax[0].set_xlim([0.3,1]);ax[0].set_ylim([0,1])

ps2 = ax[1].pcolormesh(crh_bin,cldfrac_bin,hist_bin_cfp.T[:-1,:-1]*24,cmap='jet',vmin=5)
cbar = plt.colorbar(ps2,ax=ax[1],shrink=0.8)
cbar.set_label('Precipitation [mm/d]',fontsize=13)
ax[1].contour(crh_bin,cldfrac_bin,hist_bin_cfp.T,level=[10,20,30],colors=['w'])
ax[1].set_xlabel('Column relative humidity',fontsize=14);ax[1].set_ylabel('Cloud cover',fontsize=14)
ax[1].set_xlim([0.3,1]);ax[1].set_ylim([0,1])

plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(12,5))
ps1 = ax[0].pcolormesh(crh_bin,Abar_bin,np.log10(hist_bin_A.T/np.sum(hist_bin_A))[:-1,:-1])
cbar = plt.colorbar(ps1,ax=ax[0],shrink=0.8)
cbar.set_label('Log$_{10}$(Probability)',fontsize=14)
ax[0].set_xlabel('Column relative humidity',fontsize=13);ax[0].set_ylabel(r'$\sqrt{\bar{A}}$  [km]',fontsize=14)
ax[0].set_xlim([0.3,1]);ax[0].set_ylim([0,400])

ps2 = ax[1].pcolormesh(crh_bin,Abar_bin,hist_bin_Ap.T[:-1,:-1]*24,cmap='jet',vmin=5)
cbar = plt.colorbar(ps2,ax=ax[1],shrink=0.8)
cbar.set_label('Precipitation [mm/d]',fontsize=13)
ax[1].contour(crh_bin,Abar_bin,hist_bin_Ap.T,level=[10,20,30],colors=['w'])
ax[1].set_xlabel('Column relative humidity',fontsize=14);ax[1].set_ylabel(r'$\sqrt{\bar{A}}$  [km]',fontsize=14)
ax[1].set_xlim([0.3,1]);ax[1].set_ylim([0,400])

plt.tight_layout()

### Grouping by cldfrac and Abar

In [ ]:
def idx_group(var1_all,v1_lbound,v1_ubound,var2_all,v2_lbound,v2_ubound):
    
    idx1 = np.where(np.logical_and(var1_all[:,24]>=v1_lbound,var1_all[:,24]<v1_ubound))[0]
    idx2 = np.where(np.logical_and(var2_all[:,24]>=v2_lbound,var2_all[:,24]<v2_ubound))[0]
    idx = np.intersect1d(idx1,idx2)
    
    return idx

In [ ]:
def qvint_layer(q_all,T_all,lev,lp,up):
    
    # partial cwv
    idx = np.where(np.logical_and(lev<=lp,lev>up))[0]
    q_layer = q_all[:,:,idx]
    T_layer = T_all[:,:,idx]
    
    lev_layer = np.tile(lev[idx],(T_layer.shape[0],T_layer.shape[1]))
    lev_layer = (lev_layer.reshape(T_layer.shape[0],T_layer.shape[1],len(lev[idx])))
        
    ro_layer = lev_layer*100/(287.15*T_layer)
    vint_layer = -np.trapz(q_layer/ro_layer,lev_layer*100,axis=2)/9.8 # vertical integral of q    
    
    return vint_layer

In [ ]:
# event separation by cloud cover (convective scale) and CRH 
idx_G1 = idx_group(cldfrac_all,0,0.1,crh_all,0,1)
idx_G2 = idx_group(cldfrac_all,0.1,0.2,crh_all,0,1)
idx_G3 = idx_group(cldfrac_all,0.2,0.4,crh_all,0,1)
idx_G4 = idx_group(cldfrac_all,0.4,0.6,crh_all,0,1)
idx_G5 = idx_group(cldfrac_all,0.6,0.8,crh_all,0,1)

In [ ]:
idx_G1AM = idx_group(Abar_all,0,100,crh_all,0.7,1)
idx_G2AM = idx_group(Abar_all,100,200,crh_all,0.7,1)
idx_G3AM = idx_group(Abar_all,200,300,crh_all,0.7,1)
idx_G4AM = idx_group(Abar_all,300,600,crh_all,0.7,1)

idx_G1AD = idx_group(Abar_all,0,100,crh_all,0,0.7)
idx_G2AD = idx_group(Abar_all,100,200,crh_all,0,0.7)
idx_G3AD = idx_group(Abar_all,200,300,crh_all,0,0.7)
idx_G4AD = idx_group(Abar_all,300,600,crh_all,0,0.7)

In [ ]:
qvint_G1_BL,qvint_G1_LF,qvint_G1_MF = (qvint_layer(q_all[idx_G1,:,:],T_all[idx_G1,:,:],lev,1000,900),
                                       qvint_layer(q_all[idx_G1,:,:],T_all[idx_G1,:,:],lev,900,700),
                                       qvint_layer(q_all[idx_G1,:,:],T_all[idx_G1,:,:],lev,700,500))

qvint_G2_BL,qvint_G2_LF,qvint_G2_MF = (qvint_layer(q_all[idx_G2,:,:],T_all[idx_G2,:,:],lev,1000,900),
                                       qvint_layer(q_all[idx_G2,:,:],T_all[idx_G2,:,:],lev,900,700),
                                       qvint_layer(q_all[idx_G2,:,:],T_all[idx_G2,:,:],lev,700,500))

qvint_G3_BL,qvint_G3_LF,qvint_G3_MF = (qvint_layer(q_all[idx_G3,:,:],T_all[idx_G3,:,:],lev,1000,900),
                                       qvint_layer(q_all[idx_G3,:,:],T_all[idx_G3,:,:],lev,900,700),
                                       qvint_layer(q_all[idx_G3,:,:],T_all[idx_G3,:,:],lev,700,500))

qvint_G4_BL,qvint_G4_LF,qvint_G4_MF = (qvint_layer(q_all[idx_G4,:,:],T_all[idx_G4,:,:],lev,1000,900),
                                       qvint_layer(q_all[idx_G4,:,:],T_all[idx_G4,:,:],lev,900,700),
                                       qvint_layer(q_all[idx_G4,:,:],T_all[idx_G4,:,:],lev,700,500))

qvint_G5_BL,qvint_G5_LF,qvint_G5_MF = (qvint_layer(q_all[idx_G5,:,:],T_all[idx_G5,:,:],lev,1000,900),
                                       qvint_layer(q_all[idx_G5,:,:],T_all[idx_G5,:,:],lev,900,700),
                                       qvint_layer(q_all[idx_G5,:,:],T_all[idx_G5,:,:],lev,700,500))

In [ ]:
fig = plt.figure(figsize=(6,5))
plt.plot(np.arange(-72,75,3),np.nanmean(cwv_all[idx_G1,:],axis=0),'xk',markersize=8,
        markerfacecolor='None')
plt.plot(np.arange(-72,75,3),np.nanmean(cwv_all[idx_G2,:],axis=0),'or',markersize=8,
        markerfacecolor='None')
plt.plot(np.arange(-72,75,3),np.nanmean(cwv_all[idx_G3,:],axis=0),'^g',markersize=8,
        markerfacecolor='None')
plt.plot(np.arange(-72,75,3),np.nanmean(cwv_all[idx_G4,:],axis=0),'db',markersize=8,
        markerfacecolor='None')
plt.plot(np.arange(-72,75,3),np.nanmean(cwv_all[idx_G5,:],axis=0),'sm',markersize=8,
        markerfacecolor='None')

plt.hlines(y=np.nanmean(cwv_all[idx_G1,:],axis=0)[0],xmin=-72,xmax=72
           ,color='r',alpha=0.4)
plt.hlines(y=np.nanmean(cwv_all[idx_G2,:],axis=0)[0],xmin=-72,xmax=72
           ,color='g',alpha=0.4)
plt.hlines(y=np.nanmean(cwv_all[idx_G3,:],axis=0)[0],xmin=-72,xmax=72
           ,color='b',alpha=0.4)
plt.hlines(y=np.nanmean(cwv_all[idx_G4,:],axis=0)[0],xmin=-72,xmax=72
           ,color='m',alpha=0.4)
plt.hlines(y=np.nanmean(cwv_all[idx_G5,:],axis=0)[0],xmin=-72,xmax=72
           ,color='k',alpha=0.4)

plt.vlines(x=0,ymin=40,ymax=70,linestyle=':');plt.ylim([40,70])
plt.legend(['0-0.1','0.1-0.2','0.2-0.4','0.4-0.6','0.6-0.8'])
plt.xlabel('hour',fontsize=14);plt.ylabel('CWV [mm]',fontsize=14)

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))
ax[0].plot(np.arange(-72,75,3),np.nanmean(qvint_G1_BL,axis=0),'or',markersize=6,
        markerfacecolor='None')
ax[0].plot(np.arange(-72,75,3),np.nanmean(qvint_G2_BL,axis=0),'og',markersize=6,
        markerfacecolor='None')
ax[0].plot(np.arange(-72,75,3),np.nanmean(qvint_G3_BL,axis=0),'ob',markersize=6,
        markerfacecolor='None')
ax[0].plot(np.arange(-72,75,3),np.nanmean(qvint_G4_BL,axis=0),'om',markersize=6,
        markerfacecolor='None')
ax[0].vlines(x=0,ymin=0,ymax=50,linestyles=':');ax[0].set_ylim([8,16])

ax[1].plot(np.arange(-72,75,3),np.nanmean(qvint_G1_LF,axis=0),'or',markersize=6,
        markerfacecolor='None')
ax[1].plot(np.arange(-72,75,3),np.nanmean(qvint_G2_LF,axis=0),'og',markersize=6,
        markerfacecolor='None')
ax[1].plot(np.arange(-72,75,3),np.nanmean(qvint_G3_LF,axis=0),'ob',markersize=6,
        markerfacecolor='None')
ax[1].plot(np.arange(-72,75,3),np.nanmean(qvint_G4_LF,axis=0),'om',markersize=6,
        markerfacecolor='None')
ax[1].vlines(x=0,ymin=0,ymax=50,linestyles=':');ax[1].set_ylim([15,23])

ax[2].plot(np.arange(-72,75,3),np.nanmean(qvint_G1_MF,axis=0),'or',markersize=6,
        markerfacecolor='None')
ax[2].plot(np.arange(-72,75,3),np.nanmean(qvint_G2_MF,axis=0),'og',markersize=6,
        markerfacecolor='None')
ax[2].plot(np.arange(-72,75,3),np.nanmean(qvint_G3_MF,axis=0),'ob',markersize=6,
        markerfacecolor='None')
ax[2].plot(np.arange(-72,75,3),np.nanmean(qvint_G4_MF,axis=0),'om',markersize=6,
        markerfacecolor='None')
ax[2].vlines(x=0,ymin=0,ymax=50,linestyles=':');ax[2].set_ylim([7,15])

In [ ]:
# qvflux vertical evolution

In [ ]:
qvint_G1_BL,qvint_G1_LF,qvint_G1_MF = (qvint_layer(q_all[idx_G1AD,:,:],T_all[idx_G1AD,:,:],lev,1000,900),
                                       qvint_layer(q_all[idx_G1AD,:,:],T_all[idx_G1AD,:,:],lev,900,700),
                                       qvint_layer(q_all[idx_G1AD,:,:],T_all[idx_G1AD,:,:],lev,700,500))

qvint_G2_BL,qvint_G2_LF,qvint_G2_MF = (qvint_layer(q_all[idx_G2AD,:,:],T_all[idx_G2AD,:,:],lev,1000,900),
                                       qvint_layer(q_all[idx_G2AD,:,:],T_all[idx_G2AD,:,:],lev,900,700),
                                       qvint_layer(q_all[idx_G2AD,:,:],T_all[idx_G2AD,:,:],lev,700,500))

qvint_G3_BL,qvint_G3_LF,qvint_G3_MF = (qvint_layer(q_all[idx_G3AD,:,:],T_all[idx_G3AD,:,:],lev,1000,900),
                                       qvint_layer(q_all[idx_G3AD,:,:],T_all[idx_G3AD,:,:],lev,900,700),
                                       qvint_layer(q_all[idx_G3AD,:,:],T_all[idx_G3AD,:,:],lev,700,500))

qvint_G4_BL,qvint_G4_LF,qvint_G4_MF = (qvint_layer(q_all[idx_G4AD,:,:],T_all[idx_G4AD,:,:],lev,1000,900),
                                       qvint_layer(q_all[idx_G4AD,:,:],T_all[idx_G4AD,:,:],lev,900,700),
                                       qvint_layer(q_all[idx_G4AD,:,:],T_all[idx_G4AD,:,:],lev,700,500))

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))
ax[0].plot(np.nanmean(qvint_G1_BL,axis=0),'-or')
ax[0].plot(np.nanmean(qvint_G2_BL,axis=0),'-og')
ax[0].plot(np.nanmean(qvint_G3_BL,axis=0),'-ob')
ax[0].plot(np.nanmean(qvint_G4_BL,axis=0),'-om')
#ax[0].vlines(x=24,ymin=0,ymax=50,linestyles=':');ax[0].set_ylim([10.9,11.4])

ax[1].plot(np.nanmean(qvint_G1_LF,axis=0),'-or')
ax[1].plot(np.nanmean(qvint_G2_LF,axis=0),'-og')
ax[1].plot(np.nanmean(qvint_G3_LF,axis=0),'-ob')
ax[1].plot(np.nanmean(qvint_G4_LF,axis=0),'-om')
#ax[1].vlines(x=24,ymin=0,ymax=50,linestyles=':');ax[1].set_ylim([17.5,20])

ax[2].plot(np.nanmean(qvint_G1_MF,axis=0),'-or')
ax[2].plot(np.nanmean(qvint_G2_MF,axis=0),'-og')
ax[2].plot(np.nanmean(qvint_G3_MF,axis=0),'-ob')
ax[2].plot(np.nanmean(qvint_G4_MF,axis=0),'-om')
#ax[2].vlines(x=24,ymin=0,ymax=50,linestyles=':');ax[2].set_ylim([9.5,13.5])

#### Moisture budget analysis based on cloud scale

In [ ]:
# column-integrated moisture flux 
qvflux_vint_all = np.trapz(qvflux_all,lev*100,axis=2)/9.8

In [ ]:
# extend SFX array
LHF_allre = np.zeros((len(time_event),49))
for n in range(len(time_event)):
    LHF_allre[n,:48] = LHF_all[n,:]
    LHF_allre[n,48] = LHF_all[n,-1]

In [ ]:
TQLI_all.shape

In [ ]:
# TQLI local tendency
TQLI_tend = np.gradient(TQLI_all,3,axis=1)

In [ ]:
# SFX grouped by idx_AM, area-binned, moist 
fig,ax = plt.subplots(5,1,figsize=(10,10))
ax[0].plot(np.linspace(-72,72,49),np.nanmean(LHF_allre[idx_G1,:]/2.5e6*86400,axis=0),'-g',linewidth=1.5)
ax[1].plot(np.linspace(-72,72,49),np.nanmean(LHF_allre[idx_G2,:]/2.5e6*86400,axis=0),'-g',linewidth=1.5)
ax[2].plot(np.linspace(-72,72,49),np.nanmean(LHF_allre[idx_G3,:]/2.5e6*86400,axis=0),'-g',linewidth=1.5)
ax[3].plot(np.linspace(-72,72,49),np.nanmean(LHF_allre[idx_G4,:]/2.5e6*86400,axis=0),'-g',linewidth=1.5)
ax[4].plot(np.linspace(-72,72,49),np.nanmean(LHF_allre[idx_G5,:]/2.5e6*86400,axis=0),'-g',linewidth=1.5)

ax[0].plot(np.linspace(-72,72,49),np.nanmean(qvflux_vint_all[idx_G1,:]*86400,axis=0),color='navy',linewidth=1.5)
ax[1].plot(np.linspace(-72,72,49),np.nanmean(qvflux_vint_all[idx_G2,:]*86400,axis=0),color='navy',linewidth=1.5)
ax[2].plot(np.linspace(-72,72,49),np.nanmean(qvflux_vint_all[idx_G3,:]*86400,axis=0),color='navy',linewidth=1.5)
ax[3].plot(np.linspace(-72,72,49),np.nanmean(qvflux_vint_all[idx_G4,:]*86400,axis=0),color='navy',linewidth=1.5)
ax[4].plot(np.linspace(-72,72,49),np.nanmean(qvflux_vint_all[idx_G5,:]*86400,axis=0),color='navy',linewidth=1.5)

ax[0].plot(np.linspace(-72,72,49),np.nanmean(-prec_all[idx_G1,:]*24,axis=0),color='darkorange',linewidth=1.5)
ax[1].plot(np.linspace(-72,72,49),np.nanmean(-prec_all[idx_G2,:]*24,axis=0),color='darkorange',linewidth=1.5)
ax[2].plot(np.linspace(-72,72,49),np.nanmean(-prec_all[idx_G3,:]*24,axis=0),color='darkorange',linewidth=1.5)
ax[3].plot(np.linspace(-72,72,49),np.nanmean(-prec_all[idx_G4,:]*24,axis=0),color='darkorange',linewidth=1.5)
ax[4].plot(np.linspace(-72,72,49),np.nanmean(-prec_all[idx_G5,:]*24,axis=0),color='darkorange',linewidth=1.5)

# net mositure storage based on the budget equation
net1 = np.nanmean(LHF_allre[idx_G1,:]/2.5e6*86400,axis=0)+ \
             np.nanmean(qvflux_vint_all[idx_G1,:]*86400,axis=0)- \
             np.nanmean(prec_all[idx_G1,:]*24,axis=0)
net2 = np.nanmean(LHF_allre[idx_G2,:]/2.5e6*86400,axis=0)+ \
             np.nanmean(qvflux_vint_all[idx_G2,:]*86400,axis=0)- \
             np.nanmean(prec_all[idx_G2,:]*24,axis=0)
net3 = np.nanmean(LHF_allre[idx_G3,:]/2.5e6*86400,axis=0)+ \
             np.nanmean(qvflux_vint_all[idx_G3,:]*86400,axis=0)- \
             np.nanmean(prec_all[idx_G3,:]*24,axis=0)
net4 = np.nanmean(LHF_allre[idx_G4,:]/2.5e6*86400,axis=0)+ \
             np.nanmean(qvflux_vint_all[idx_G4,:]*86400,axis=0)- \
             np.nanmean(prec_all[idx_G4,:]*24,axis=0)
net5 = np.nanmean(LHF_allre[idx_G5,:]/2.5e6*86400,axis=0)+ \
             np.nanmean(qvflux_vint_all[idx_G5,:]*86400,axis=0)- \
             np.nanmean(prec_all[idx_G5,:]*24,axis=0)

ax[0].fill_between(np.linspace(-72,72,49),net1, 0, where=net1>0, facecolor='b',interpolate=True,alpha=0.4)
ax[0].fill_between(np.linspace(-72,72,49),net1, 0, where=net1<0, facecolor='r',interpolate=True,alpha=0.4)
ax[1].fill_between(np.linspace(-72,72,49),net2, 0, where=net2>0, facecolor='b',interpolate=True,alpha=0.4)
ax[1].fill_between(np.linspace(-72,72,49),net2, 0, where=net2<0, facecolor='r',interpolate=True,alpha=0.4)
ax[2].fill_between(np.linspace(-72,72,49),net3, 0, where=net3>0, facecolor='b',interpolate=True,alpha=0.4)
ax[2].fill_between(np.linspace(-72,72,49),net3, 0, where=net3<0, facecolor='r',interpolate=True,alpha=0.4)
ax[3].fill_between(np.linspace(-72,72,49),net4, 0, where=net4>0, facecolor='b',interpolate=True,alpha=0.4)
ax[3].fill_between(np.linspace(-72,72,49),net4, 0, where=net4<0, facecolor='r',interpolate=True,alpha=0.4)
ax[4].fill_between(np.linspace(-72,72,49),net5, 0, where=net5>0, facecolor='b',interpolate=True,alpha=0.4)
ax[4].fill_between(np.linspace(-72,72,49),net5, 0, where=net5<0, facecolor='r',interpolate=True,alpha=0.4)

ax[0].set_xlim([-72,72]);ax[0].hlines(y=0,xmin=-72,xmax=600,linewidth=0.5,zorder=3)
ax[1].set_xlim([-72,72]);ax[1].hlines(y=0,xmin=-72,xmax=600,linewidth=0.5,zorder=3)
ax[2].set_xlim([-72,72]);ax[2].hlines(y=0,xmin=-72,xmax=600,linewidth=0.5,zorder=3)
ax[3].set_xlim([-72,72]);ax[3].hlines(y=0,xmin=-72,xmax=600,linewidth=0.5,zorder=3)
ax[4].set_xlim([-72,72]);ax[4].hlines(y=0,xmin=-72,xmax=600,linewidth=0.5,zorder=3)

ax[0].set_ylim([-10,10]);ax[1].set_ylim([-15,15]);ax[2].set_ylim([-25,25])
ax[3].set_ylim([-40,40]);ax[4].set_ylim([-50,50])

ax[0].set_ylabel('mm/day',fontsize=12)
ax[1].set_ylabel('mm/day',fontsize=12)
ax[2].set_ylabel('mm/day',fontsize=12)
ax[3].set_ylabel('mm/day',fontsize=12)
ax[4].set_ylabel('mm/day',fontsize=12)
ax[4].set_xlabel('hour',fontsize=12)

ax[0].grid(linestyle='--',color='grey',linewidth=0.2);ax[0].set_xticks(np.arange(-72,84,12))
ax[1].grid(linestyle='--',color='grey',linewidth=0.2);ax[1].set_xticks(np.arange(-72,84,12))
ax[2].grid(linestyle='--',color='grey',linewidth=0.2);ax[2].set_xticks(np.arange(-72,84,12))
ax[3].grid(linestyle='--',color='grey',linewidth=0.2);ax[3].set_xticks(np.arange(-72,84,12))
ax[4].grid(linestyle='--',color='grey',linewidth=0.2);ax[4].set_xticks(np.arange(-72,84,12))

ax[0].set_xticklabels(labels=[])
ax[1].set_xticklabels(labels=[])
ax[2].set_xticklabels(labels=[])
ax[3].set_xticklabels(labels=[])

ax[0].set_title('Column-integrated moisture budget analysis, Qv_ten = QF + LHF - P',fontsize=13)
ax[0].legend(['LHF','MFlux','Prec'], loc=1,
       ncol=3, borderaxespad=0.2, frameon=False)
plt.show()

In [ ]:
# SFX grouped by idx_AM, area-binned, moist 
fig,ax = plt.subplots(5,1,figsize=(10,10))

# net mositure storage based on the budget equation
net1 = np.nanmean(SFX_allre[idx_G1,:]/2.5e6*86400,axis=0)+ \
             np.nanmean(qvflux_vint_all[idx_G1,:]*86400,axis=0)- \
             np.nanmean(prec_all[idx_G1,:]*24,axis=0)
net2 = np.nanmean(SFX_allre[idx_G2,:]/2.5e6*86400,axis=0)+ \
             np.nanmean(qvflux_vint_all[idx_G2,:]*86400,axis=0)- \
             np.nanmean(prec_all[idx_G2,:]*24,axis=0)
net3 = np.nanmean(SFX_allre[idx_G3,:]/2.5e6*86400,axis=0)+ \
             np.nanmean(qvflux_vint_all[idx_G3,:]*86400,axis=0)- \
             np.nanmean(prec_all[idx_G3,:]*24,axis=0)
net4 = np.nanmean(SFX_allre[idx_G4,:]/2.5e6*86400,axis=0)+ \
             np.nanmean(qvflux_vint_all[idx_G4,:]*86400,axis=0)- \
             np.nanmean(prec_all[idx_G4,:]*24,axis=0)
net5 = np.nanmean(SFX_allre[idx_G5,:]/2.5e6*86400,axis=0)+ \
             np.nanmean(qvflux_vint_all[idx_G5,:]*86400,axis=0)- \
             np.nanmean(prec_all[idx_G5,:]*24,axis=0)

ax[0].fill_between(np.linspace(-72,72,49),net1, 0, where=net1>0, facecolor='b',interpolate=True,alpha=0.4)
ax[0].fill_between(np.linspace(-72,72,49),net1, 0, where=net1<0, facecolor='r',interpolate=True,alpha=0.4)
ax[1].fill_between(np.linspace(-72,72,49),net2, 0, where=net2>0, facecolor='b',interpolate=True,alpha=0.4)
ax[1].fill_between(np.linspace(-72,72,49),net2, 0, where=net2<0, facecolor='r',interpolate=True,alpha=0.4)
ax[2].fill_between(np.linspace(-72,72,49),net3, 0, where=net3>0, facecolor='b',interpolate=True,alpha=0.4)
ax[2].fill_between(np.linspace(-72,72,49),net3, 0, where=net3<0, facecolor='r',interpolate=True,alpha=0.4)
ax[3].fill_between(np.linspace(-72,72,49),net4, 0, where=net4>0, facecolor='b',interpolate=True,alpha=0.4)
ax[3].fill_between(np.linspace(-72,72,49),net4, 0, where=net4<0, facecolor='r',interpolate=True,alpha=0.4)
ax[4].fill_between(np.linspace(-72,72,49),net5, 0, where=net5>0, facecolor='b',interpolate=True,alpha=0.4)
ax[4].fill_between(np.linspace(-72,72,49),net5, 0, where=net5<0, facecolor='r',interpolate=True,alpha=0.4)

ax[0].set_xlim([-72,72]);ax[0].hlines(y=0,xmin=-72,xmax=600,linewidth=0.5,zorder=3)
ax[1].set_xlim([-72,72]);ax[1].hlines(y=0,xmin=-72,xmax=600,linewidth=0.5,zorder=3)
ax[2].set_xlim([-72,72]);ax[2].hlines(y=0,xmin=-72,xmax=600,linewidth=0.5,zorder=3)
ax[3].set_xlim([-72,72]);ax[3].hlines(y=0,xmin=-72,xmax=600,linewidth=0.5,zorder=3)
ax[4].set_xlim([-72,72]);ax[4].hlines(y=0,xmin=-72,xmax=600,linewidth=0.5,zorder=3)

ax[0].set_ylim([-10,10]);ax[1].set_ylim([-10,10]);ax[2].set_ylim([-10,10])
ax[3].set_ylim([-10,10]);ax[4].set_ylim([-10,10])

ax[0].set_ylabel('mm/day',fontsize=12)
ax[1].set_ylabel('mm/day',fontsize=12)
ax[2].set_ylabel('mm/day',fontsize=12)
ax[3].set_ylabel('mm/day',fontsize=12)
ax[4].set_ylabel('mm/day',fontsize=12)
ax[4].set_xlabel('hour',fontsize=12)

ax[0].grid(linestyle='--',color='grey',linewidth=0.2);ax[0].set_xticks(np.arange(-72,84,12))
ax[1].grid(linestyle='--',color='grey',linewidth=0.2);ax[1].set_xticks(np.arange(-72,84,12))
ax[2].grid(linestyle='--',color='grey',linewidth=0.2);ax[2].set_xticks(np.arange(-72,84,12))
ax[3].grid(linestyle='--',color='grey',linewidth=0.2);ax[3].set_xticks(np.arange(-72,84,12))
ax[4].grid(linestyle='--',color='grey',linewidth=0.2);ax[4].set_xticks(np.arange(-72,84,12))

ax[0].set_xticklabels(labels=[])
ax[1].set_xticklabels(labels=[])
ax[2].set_xticklabels(labels=[])
ax[3].set_xticklabels(labels=[])

ax[0].set_title('Column-integrated moisture budget analysis, Qv_ten = QF + LHF - P',fontsize=13)
ax[0].legend(['local tendency'], loc=1,
       ncol=1, borderaxespad=0.2, frameon=False)
plt.show()

In [ ]:
# SFX grouped by idx_AM, area-binned, moist 
fig,ax = plt.subplots(2,2,figsize=(8,6))
ax[0,0].plot(np.nanmean(SFX_allre[idx_G1,:],axis=0),'--k',linewidth=3)
ax[0,1].plot(np.nanmean(SFX_allre[idx_G2,:],axis=0),'--k',linewidth=3)
ax[1,0].plot(np.nanmean(SFX_allre[idx_G3,:],axis=0),'--k',linewidth=3)
ax[1,1].plot(np.nanmean(SFX_allre[idx_G4,:],axis=0),'--k',linewidth=3)
ax[1,1].plot(np.nanmean(SFX_allre[idx_G5,:],axis=0),'--b',linewidth=3)


# ax[0,0].plot(np.nanmean(2.5e6*qvflux_vint_all[idx_G1AD,:],axis=0),'-k',linewidth=3)
# ax[0,1].plot(np.nanmean(2.5e6*qvflux_vint_all[idx_G2AD,:],axis=0),'-k',linewidth=3)
# ax[1,0].plot(np.nanmean(2.5e6*qvflux_vint_all[idx_G3AD,:],axis=0),'-k',linewidth=3)
# ax[1,1].plot(np.nanmean(2.5e6*qvflux_vint_all[idx_G4AD,:],axis=0),'-k',linewidth=3)

#ax[0,0].set_ylim([-30,250]);ax[0,0].vlines(x=24,ymin=-30,ymax=600,linestyle=':')
#ax[0,1].set_ylim([-30,350]);ax[0,1].vlines(x=24,ymin=-30,ymax=600,linestyle=':')
#ax[1,0].set_ylim([-30,450]);ax[1,0].vlines(x=24,ymin=-30,ymax=600,linestyle=':')
#ax[1,1].set_ylim([-30,550]);ax[1,1].vlines(x=24,ymin=-30,ymax=600,linestyle=':')
#ax[0].vlines(x=24,ymin=0,ymax=50,linestyles=':');ax[0].set_ylim([10.9,11.4])

In [ ]:
plt.plot(np.arange(-72,75,3),np.nanmean(crh_q4_all[idx_G1,:],axis=0),'r')
plt.plot(np.arange(-72,75,3),np.nanmean(crh_q4_all[idx_G2,:],axis=0),'g')
plt.plot(np.arange(-72,75,3),np.nanmean(crh_q4_all[idx_G3,:],axis=0),'b')
plt.plot(np.arange(-72,75,3),np.nanmean(crh_q4_all[idx_G4,:],axis=0),'k')
plt.plot(np.arange(-72,75,3),np.nanmean(crh_q4_all[idx_G5,:],axis=0),'m')

plt.plot(np.arange(-72,75,3),np.nanmean(crh_q1_all[idx_G1,:],axis=0),'--r')
plt.plot(np.arange(-72,75,3),np.nanmean(crh_q1_all[idx_G2,:],axis=0),'--g')
plt.plot(np.arange(-72,75,3),np.nanmean(crh_q1_all[idx_G3,:],axis=0),'--b')
plt.plot(np.arange(-72,75,3),np.nanmean(crh_q1_all[idx_G4,:],axis=0),'--k')
plt.plot(np.arange(-72,75,3),np.nanmean(crh_q1_all[idx_G5,:],axis=0),'--m')

In [ ]:
plt.plot(np.arange(-72,75,3),np.nanmean(crh_q4_all[idx_G1,:]-crh_q1_all[idx_G1,:],axis=0),'r')
plt.plot(np.arange(-72,75,3),np.nanmean(crh_q4_all[idx_G2,:]-crh_q1_all[idx_G2,:],axis=0),'g')
plt.plot(np.arange(-72,75,3),np.nanmean(crh_q4_all[idx_G3,:]-crh_q1_all[idx_G3,:],axis=0),'b')
plt.plot(np.arange(-72,75,3),np.nanmean(crh_q4_all[idx_G4,:]-crh_q1_all[idx_G4,:],axis=0),'k')
plt.plot(np.arange(-72,75,3),np.nanmean(crh_q4_all[idx_G5,:]-crh_q1_all[idx_G5,:],axis=0),'m')

####  successive events after different scale of convection formation

In [ ]:
idx_G4[:10]

In [ ]:
def var_successive(idx_group,succ_num,var_all):
    var_com = np.zeros((len(idx_group),succ_num)) # variable stored
    days_com = np.copy(var_com)
    for n,idx in enumerate(idx_group):

        target_time = time_event[idx]
        target_lat = lat_event[idx]
        target_lon = lon_event[idx]

        idx1 = np.where(lat_event == target_lat)
        idx2 = np.where(lon_event == target_lon)
        idx_pos = np.intersect1d(idx1,idx2)

        for i,idy in enumerate(idx_pos):
            if target_time == time_event[idy]:
                idx_succ = idx_pos[i:i+succ_num]
                var_com[n,:len(idx_succ)] = var_all[idx_succ,24]
                #days_com[n,:len(idx_succ)] = time_event[idx_succ]
                
    return var_com #days_com

In [ ]:
(time_event[1:5]-time_event[0:4])

In [ ]:
cldfrac_succG1 = var_successive(idx_G1,5,crh_all)
cldfrac_succG2 = var_successive(idx_G2,5,crh_all)
cldfrac_succG3 = var_successive(idx_G3,5,crh_all)
cldfrac_succG4 = var_successive(idx_G4,5,crh_all)
cldfrac_succG5 = var_successive(idx_G5,5,crh_all)

plt.plot(np.nanmean(cldfrac_succG1,axis=0),'-or')
plt.plot(np.nanmean(cldfrac_succG2,axis=0),'-og')
plt.plot(np.nanmean(cldfrac_succG3,axis=0),'-ob')
plt.plot(np.nanmean(cldfrac_succG4,axis=0),'-om')
plt.plot(np.nanmean(cldfrac_succG5,axis=0),'-ok')

In [ ]:
from scipy.ndimage import gaussian_filter

In [ ]:
plt.plot(np.arange(-72,75,3),gaussian_filter(np.nanmean(np.gradient(cwv_all[idx_G1,:],2,axis=1)*24,axis=0),1),'r')
plt.plot(np.arange(-72,75,3),gaussian_filter(np.nanmean(np.gradient(cwv_all[idx_G2,:],2,axis=1)*24,axis=0),1),'g')
plt.plot(np.arange(-72,75,3),gaussian_filter(np.nanmean(np.gradient(cwv_all[idx_G3,:],2,axis=1)*24,axis=0),1),'b')
plt.plot(np.arange(-72,75,3),gaussian_filter(np.nanmean(np.gradient(cwv_all[idx_G4,:],2,axis=1)*24,axis=0),1),'b')
#plt.plot(np.arange(-72,75,3),np.nanmean(np.gradient(cwv_all[idx_G5,:],3,axis=1)*24,axis=0),'-ok')

plt.xlim([-72,72]);plt.grid()

In [ ]:
plt.plot(np.arange(-72,75,3),gaussian_filter(np.nanmean(np.gradient(prec_all[idx_G1,:],3,axis=1)*24,axis=0),2),'-r')
plt.plot(np.arange(-72,75,3),gaussian_filter(np.nanmean(np.gradient(prec_all[idx_G2,:],3,axis=1)*24,axis=0),2),'-g')
plt.plot(np.arange(-72,75,3),gaussian_filter(np.nanmean(np.gradient(prec_all[idx_G3,:],3,axis=1)*24,axis=0),2),'-b')
plt.plot(np.arange(-72,75,3),gaussian_filter(np.nanmean(np.gradient(prec_all[idx_G4,:],3,axis=1)*24,axis=0),2),'-k')

plt.xlim([-48,48]);plt.grid()

In [ ]:
plt.plot(np.arange(-72,75,3),np.nanmean(np.gradient(crh_all[idx_G1,:],3,axis=1)*24,axis=0),'-or')
plt.plot(np.arange(-72,75,3),np.nanmean(np.gradient(crh_all[idx_G2,:],3,axis=1)*24,axis=0),'-og')
plt.plot(np.arange(-72,75,3),np.nanmean(np.gradient(crh_all[idx_G3,:],3,axis=1)*24,axis=0),'-ob')
plt.plot(np.arange(-72,75,3),np.nanmean(np.gradient(crh_all[idx_G4,:],3,axis=1)*24,axis=0))

plt.xlim([-48,48]);plt.grid()

In [ ]:
plt.plot(np.arange(-72,75,3),gaussian_filter(np.nanmean(prec_all[idx_G1,:],axis=0),0),'-r')
plt.plot(np.arange(-72,75,3),gaussian_filter(np.nanmean(prec_all[idx_G2,:],axis=0),0),'-g')
plt.plot(np.arange(-72,75,3),gaussian_filter(np.nanmean(prec_all[idx_G3,:],axis=0),0),'-b')
plt.plot(np.arange(-72,75,3),gaussian_filter(np.nanmean(prec_all[idx_G4,:],axis=0),0),'-k')

In [ ]:
clevs = np.linspace(-0.6,0.6,13)
q_mean = np.tile(q_all[:,:,:].mean(axis=1),49).reshape(q_all.shape)
qa_all = q_all[idx_G1AM,:,:] - q_mean[idx_G1AM,:,:] # moisture anomaly
plt.contourf(np.arange(-72,75,3),lev,np.nanmean(qa_all*1000,axis=0).T,levels=clevs)
plt.ylim([1000,100]);plt.colorbar();plt.show()

In [ ]:
qa_all = q_all[idx_G2AM,:,:] - q_mean[idx_G2AM,:,:] # moisture anomaly
plt.contourf(np.arange(-72,75,3),lev,np.nanmean(qa_all*1000,axis=0).T,levels=clevs)
plt.ylim([1000,100]);plt.colorbar();plt.show()

In [ ]:
qa_all = q_all[idx_G3AM,:,:] - q_mean[idx_G3AM,:,:] # moisture anomaly
plt.contourf(np.arange(-72,75,3),lev,np.nanmean(qa_all*1000,axis=0).T,levels=clevs)
plt.ylim([1000,100]);plt.colorbar();plt.show()

In [ ]:
qa_all = q_all[idx_G4AM,:,:] - q_mean[idx_G4AM,:,:] # moisture anomaly
plt.contourf(np.arange(-72,75,3),lev,np.nanmean(qa_all*1000,axis=0).T,levels=clevs)
plt.ylim([1000,100]);plt.colorbar();plt.show()

In [ ]:
#div_mean = np.tile(div_all[:,:,:].mean(axis=1),49).reshape(div_all.shape)
diva_all = div_all[idx_G1AM,:,:] #- div_mean[idx_G1,:,:] # moisture anomaly
plt.contourf(np.arange(-72,75,3),lev,np.nanmean(div_all[idx_G1AM,:],axis=0).T)
plt.ylim([1000,100]);plt.colorbar();plt.show()

In [ ]:
diva_all = div_all[idx_G2AM,:,:] #- div_mean[idx_G1,:,:] # moisture anomaly
plt.contourf(np.arange(-72,75,3),lev,np.nanmean(div_all[idx_G2AM,:],axis=0).T)
plt.ylim([1000,100]);plt.colorbar();plt.show()

In [ ]:
diva_all = div_all[idx_G3AM,:,:] #- div_mean[idx_G1,:,:] # moisture anomaly
plt.contourf(np.arange(-72,75,3),lev,np.nanmean(div_all[idx_G3AM,:],axis=0).T)
plt.ylim([1000,100]);plt.colorbar();plt.show()

In [ ]:
diva_all = div_all[idx_G4AM,:,:] #- div_mean[idx_G1,:,:] # moisture anomaly
plt.contourf(np.arange(-72,75,3),lev,np.nanmean(div_all[idx_G4AM,:],axis=0).T)
plt.ylim([1000,100]);plt.colorbar();plt.show()

In [ ]:
#omegaa_all = omega_all[idx_G2,:,:] - omega_mean[idx_G2,:,:] # moisture anomaly
plt.contourf(np.arange(-72,75,3),lev,np.nanmean(omega_all[idx_G2,:],axis=0).T)
plt.ylim([1000,100]);plt.colorbar();plt.show()

In [ ]:
#omegaa_all = omega_all[idx_G3,:,:] - omega_mean[idx_G3,:,:] # moisture anomaly
plt.contourf(np.arange(-72,75,3),lev,np.nanmean(omega_all[idx_G3,:],axis=0).T)
plt.ylim([1000,100]);plt.colorbar();plt.show()

### Omega vertical mode decomposition [Masunaga and L'ecuyer]

In [ ]:
def vertmode_omega(omega_data,omega_bg,fbc_mtx):
    'omega_bg: background omega profile, defined as paper'
    m_dim = fbc_mtx.shape[0]
    t_dim = omega_data.shape[0]
    m_time = np.zeros((m_dim,t_dim))
    omega_rcst = np.zeros(omega_data.shape) # reconstructed profile by three modes 
    for t in range(t_dim):
        omega_mtx = np.matrix((omega_data[t,:25]-omega_bg[:25]))
        m_mtx = (fbc_mtx*fbc_mtx.T).I*(fbc_mtx*omega_mtx.T) # regression coefficient    
        m_time[:,t] = m_mtx.A.T
        omega_rcst[t,:] = m_mtx.T*fbc_mtx
    return m_time, omega_rcst

In [ ]:
# prescribed vertical mode definition
ps = 1000
ptp = 100
p = lev[:25]
fbc1 = -np.sin(1*np.pi*(p-ptp)/(ps-ptp)) # prescribed first mode
fbc2 = np.sin(2*np.pi*(p-ptp)/(ps-ptp)) # prescribed second mode 
fshl = -np.sin(np.pi*(p-850)/(ps-850)); fshl[6:] = 0 # prescribed shallow mode, peak at 850hpa
fbc_mtx = np.stack((fbc1,fbc2,fshl),axis=0)
fbc_mtx = np.matrix(fbc_mtx)

In [ ]:
omega_allmn = np.nanmean(omega_all,axis=0)
omega_bg = np.nanmean(np.concatenate((omega_allmn[:4,:25],omega_allmn[-4:,:25])),axis=0)

In [ ]:
em1_time = np.zeros(prec_all.shape)
em2_time = np.zeros(prec_all.shape)
em3_time = np.zeros(prec_all.shape)

for n in range(len(omega_all)):
    m_time, omega_rcst = vertmode_omega(omega_all[n,:,:25],omega_bg,fbc_mtx)
    em1_time[n,:] = m_time[0,:]
    em2_time[n,:] = m_time[1,:]
    em3_time[n,:] = m_time[2,:]

In [ ]:
# total 
plt.plot(np.nanmean(em1_time,axis=0),'r')
plt.plot(np.nanmean(em2_time,axis=0),'b')
plt.plot(np.nanmean(em3_time,axis=0),'k')

In [ ]:
plt.plot(np.nanmean(em1_time[idx_G1,:],axis=0),'r')
plt.plot(np.nanmean(em2_time[idx_G1,:],axis=0),'b')
plt.plot(np.nanmean(em3_time[idx_G1,:],axis=0),'k')

In [ ]:
plt.plot(np.nanmean(em1_time[idx_G2,:],axis=0),'r')
plt.plot(np.nanmean(em2_time[idx_G2,:],axis=0),'b')
plt.plot(np.nanmean(em3_time[idx_G2,:],axis=0),'k')

In [ ]:
plt.plot(np.nanmean(em1_time[idx_G4,:],axis=0))
plt.plot(np.nanmean(em2_time[idx_G4,:],axis=0))
plt.plot(np.nanmean(em3_time[idx_G4,:],axis=0))

In [ ]:
cldamt_dc_all[150,:]

In [ ]:
plt.plot(np.nanmean(cldamt_cu_all[idx_G1,:],axis=0),'k')
plt.plot(np.nanmean(cldamt_st_all[idx_G1,:],axis=0),'b')
plt.plot(np.nanmean(cldamt_dc_all[idx_G1,:],axis=0),'r')

In [ ]:
num = 23
print(cldfrac_all[num,24],Abar_all[num,24],crh_all[num,24])
plt.plot(cldamt_cu_all[num,:],'k')
plt.plot(cldamt_st_all[num,:],'b')
plt.plot(cldamt_dc_all[num,:],'r')

In [ ]:
z = np.array([124,810,1541,3178,4437,5887,6000])
slp = -3/6000
u = 3 + slp*z
u

In [ ]:
-----------------------------------------------

In [ ]:
def caseview(time_event,lat_event,lon_event):

    data_sub1 = subset_MERRA2(MERRA2_3d_dir+'TQUV_3hr'
                          ,time_event,lat_event,lon_event,3,10,20)
    data_sub2 = subset_TRMM(TRMM_dir+'2014_2018'
                          ,time_event,lat_event,lon_event,3,10,20)
    data_sub3 = subset_MERRA2(MERRA2_dir+'inst_2d_hourly'
                          ,time_event,lat_event,lon_event,3,10,20)
    data_sub4 = subset_MERRA2(MERRA2_3d_dir+'omega_3hr'
                          ,time_event,lat_event,lon_event,3,10,20)
    data_sub5 = subset_gridsat(Gridsat_dir+'2014_2018'
                          ,time_event,lat_event,lon_event,3,10,20)
    
    u_sm = data_sub1.U
    v_sm = data_sub1.V
    T_sm = data_sub1.T
    q_sm = data_sub1.QV
    prec_sm = data_sub2.precipitation.resample(time='6H').nearest()
    cwv_sm = data_sub3.TQV.resample(time='6H').nearest()
    omega_sm = data_sub4.OMEGA
    bt_sm = data_sub5.irwin_cdr.resample(time='6H').nearest()

    u_950sm = u_sm.sel(lev=950).resample(time='6H').nearest()
    v_950sm = v_sm.sel(lev=950).resample(time='6H').nearest()
    u_tar = u_sm.sel(lon=slice(lon_event-5,lon_event+5),lat=slice(lat_event-5,lat_event+5))
    v_tar = v_sm.sel(lon=slice(lon_event-5,lon_event+5),lat=slice(lat_event-5,lat_event+5))
    omega_tar = omega_sm.sel(lon=slice(lon_event-5,lon_event+5),lat=slice(lat_event-5,lat_event+5))
    prec_tar = prec_sm.sel(nlon=slice(lon_event-5,lon_event+5),nlat=slice(lat_event-5,lat_event+5))
    bt_tar = bt_sm.sel(lon=slice(lon_event-5,lon_event+5),lat=slice(lat_event-5,lat_event+5))
    SCAIbt_sm,SCAI_p,Nbt_sm,N_prec = SCAI_index(prec_tar,bt_tar) # 6H data
    div_tar = np.gradient(u_tar,0.625*100*1e3,axis=3)+np.gradient(v_tar,0.5*100*1e3,axis=2) # divergence 3D
    
    p = T_sm.lev.values*units('mbar')
    T_tar = T_sm.sel(lon=slice(lon_event-5,lon_event+5),lat=slice(lat_event-5,lat_event+5))
    q_tar = q_sm.sel(lon=slice(lon_event-5,lon_event+5),lat=slice(lat_event-5,lat_event+5))
    # cape/cin calculation
    cape_sm = np.zeros(len(T_tar.time))
    cin_sm = np.copy(cape_sm)
    for t in range(len(u_tar.time)):
        T_tmp = np.nanmean(T_tar[t,:,:,:],axis=(1,2))
        q_tmp = np.nanmean(q_tar[t,:,:,:],axis=(1,2))
        T = (T_tmp-273.15)*units('degC')
        q = q_tmp*units('kg/kg')
        Td = mpcalc.dewpoint(mpcalc.vapor_pressure(p,q))
        
        try:
            Tp = mpcalc.parcel_profile(p,T[0],Td[0]).to('degC')                    
            el_pressure,el_temperature = mpcalc.el(p,T,Td) # equilibrium level
            el_idx = np.argmin(np.abs(p.magnitude - el_pressure.magnitude))                
            ELps = [el_pressure.magnitude] # Initialize an array of EL pressures for detrainment profile
            [CAPE,CIN] = mpcalc.cape_cin(p[:el_idx],T[:el_idx],Td[:el_idx],Tp[:el_idx])
            cape_sm[t] = CAPE.magnitude
            cin_sm[t] = CIN.magnitude
        except:
            cape_sm[t] = np.nan
            cin_sm[t] = np.nan
    
    # Abar calculation
    tmp = bt_tar.values
    cldfrac_sm = [len(np.where(tmp[n,:,:]<240)[0])/(tmp.shape[1]*tmp.shape[2]) for n in range(tmp.shape[0])]
    Abar_sm = cldfrac_sm/Nbt_sm*1600
    
    return (u_950sm,v_950sm,omega_sm,prec_sm,cwv_sm,bt_sm,SCAIbt_sm,Nbt_sm,Abar_sm
            ,div_tar,cape_sm,cin_sm)

In [ ]:
idx_G3AD[0:100]

In [ ]:
num=1908
u_950sm,v_950sm,omega_sm,prec_sm,cwv_sm,bt_sm,SCAIbt_sm,Nbt_sm,Abar_sm,div_sm,cape_sm,cin_sm = caseview(time_event[num],lat_event[num],lon_event[num])

In [ ]:
fig,ax = plt.subplots(7,2,figsize=(12,15))
for n in range(7):
    #ax[n] = plt.axes(projection=ccrs.PlateCarree(central_longitude=0))
    ax[n,0].pcolormesh(cwv_sm.lon,cwv_sm.lat,cwv_sm[4*n,:,:])
    ax[n,0].contour(cwv_sm.lon,cwv_sm.lat,cwv_sm[4*n,:,:],levels=[48],colors=['w'])
    ax[n,0].quiver(u_950sm.lon[::2],u_950sm.lat[::2],u_950sm[4*n,::2,::2],v_950sm[4*n,::2,::2])      
    rect1 = patches.Rectangle((lon_event[num]-5,lat_event[num]-5),10,10,linewidth=2,
                              edgecolor='r',facecolor='None')
    ax[n,0].add_patch(rect1)
    if n ==0:
        ax[n,0].set_title(time_event[num])
        
    ax[n,1].contour(prec_sm.nlon,prec_sm.nlat,prec_sm[4*n,:,:].transpose(),
                   levels=[1,5,10,30,50],linewidths=1,colors=['m'])
    tmp = bt_sm[4*n,:,:].values; #tmp[tmp>240] = np.nan
    ax[n,1].pcolormesh(bt_sm.lon,bt_sm.lat,tmp,cmap='binary_r') 
    rect2 = patches.Rectangle((lon_event[num]-5,lat_event[num]-5),10,10,linewidth=2,
                              edgecolor='r',facecolor='None')
    ax[n,1].add_patch(rect2)
    ax[n,1].text(lon_event[num]+10,lat_event[num]+7.5,'N = '+str(int(Nbt_sm[n])),color='g',fontsize=11)
    ax[n,1].text(lon_event[num]+10,lat_event[num]+5.5,'SCAI = '+str(round(SCAIbt_sm[n],1)),color='g',fontsize=11)
    ax[n,1].text(lon_event[num]+10,lat_event[num]+3.5,'Abar = '+str(round(Abar_sm[n],1)),color='g',fontsize=11)
    if n ==0:
        ax[n,1].set_title(time_event[num])

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # mm/day
SFX_bin = np.zeros((len(cldfrac_bin)-1,48)) # m/s
SFXstd_bin = np.zeros((len(cldfrac_bin)-1,48))
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    SFX_bin[n,:] = np.nanmean(SFX_all[idx,:],axis=0)
    SFXstd_bin[n,:] = np.nanmean(SFXstd_all[idx,:],axis=0)
    num_bin[n] = len(idx)
    
fig=plt.figure(figsize=(8,4))

plt.plot(np.linspace(-72,72,48,dtype=int),SFX_bin[0,:],color='k')
plt.plot(np.linspace(-72,72,48,dtype=int),SFX_bin[1,:],color='b')
plt.plot(np.linspace(-72,72,48,dtype=int),SFX_bin[2,:],color='orange')
plt.plot(np.linspace(-72,72,48,dtype=int),SFX_bin[3,:],color='r')

plt.legend(['0 < cldfrac < 0.1','0.1 < cldfrac < 0.25','0.25 < cldfrac < 0.5','0.5 < cldfrac < 1']
          ,fontsize=9)
plt.ylabel('SFX [W/m$^2$]')
plt.xlabel('Lag time [hr]')
plt.grid(axis='x',linestyle=':')
#plt.title('Precipitation evolution, 10deg box, IND')

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # mm/day
wsfc_bin = np.zeros((len(cldfrac_bin)-1,49)) # m/s
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    wsfc_bin[n,:] = np.nanmean((u_all[idx,:,0]**2+v_all[idx,:,0]**2)**0.5,axis=0)
    num_bin[n] = len(idx)
    
fig=plt.figure(figsize=(8,4))

plt.plot(np.linspace(-72,72,49,dtype=int),wsfc_bin[0,:],color='k')
plt.plot(np.linspace(-72,72,49,dtype=int),wsfc_bin[1,:],color='b')
plt.plot(np.linspace(-72,72,49,dtype=int),wsfc_bin[2,:],color='orange')
plt.plot(np.linspace(-72,72,49,dtype=int),wsfc_bin[3,:],color='r')

plt.legend(['0 < cldfrac < 0.1','0.1 < cldfrac < 0.25','0.25 < cldfrac < 0.5','0.5 < cldfrac < 1']
          ,fontsize=9)
plt.ylabel('SFX [W/m$^2$]')
plt.xlabel('Lag time [hr]')
plt.grid(axis='x',linestyle=':')
#plt.title('Precipitation evolution, 10deg box, IND')

In [ ]:
# cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # mm/day
# freq_bin = np.zeros((len(cldfrac_bin)-1,4)) # m/s

# for n in range(len(cldfrac_bin)-1):
#     idx1 = np.where(np.logical_and(cldfrac_DJF[:,24]>=cldfrac_bin[n],cldfrac_DJF[:,24]<cldfrac_bin[n+1]))[0]
#     idx2 = np.where(np.logical_and(cldfrac_MAM[:,24]>=cldfrac_bin[n],cldfrac_MAM[:,24]<cldfrac_bin[n+1]))[0]
#     idx3 = np.where(np.logical_and(cldfrac_JJA[:,24]>=cldfrac_bin[n],cldfrac_JJA[:,24]<cldfrac_bin[n+1]))[0]
#     idx4 = np.where(np.logical_and(cldfrac_SON[:,24]>=cldfrac_bin[n],cldfrac_SON[:,24]<cldfrac_bin[n+1]))[0]
    
#     freq_bin[n,0] = len(idx1)
#     freq_bin[n,1] = len(idx2)
#     freq_bin[n,2] = len(idx3)
#     freq_bin[n,3] = len(idx4)

In [ ]:
# constructing the climatology over targeted scenes for T,q,RH anomaly 
# to remove seasonality and diurnal cycle
os.chdir(MERRA2_dir+'inst_2d_hourly')
files = sorted(glob('*nc'))
data = xr.open_mfdataset(files)

In [ ]:
cwv = data.sel(lat=slice(-20,20)).TQV
cwv_clim = cwv.groupby('time.month').mean('time').compute() # climatology of daily data

In [ ]:
# test for removing seasonality over all cases
cwv_rmseason = np.copy(cwv_all)

for n in range(len(time_event)):
    cwv_climsub = cwv_clim.sel(lat=slice(lat_event[n]-5,lat_event[n]+5),
                          lon=slice(lon_event[n]-5,lon_event[n]+5),month=time_event[n].month)
    cwv_rmseason[n,:] = cwv_all[n,:] - cwv_climsub.mean().values

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # mm/day
cwva_bin = np.zeros((len(cldfrac_bin)-1,49)) # m/s
cwvastd_bin = np.zeros((len(cldfrac_bin)-1,49))
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    cwva_bin[n,:] = np.nanmean(cwv_all[idx,:],axis=0)
    cwvastd_bin[n,:] = np.nanstd(cwv_all[idx,:],axis=0)/np.sqrt(len(idx))
    num_bin[n] = len(idx)

fig=plt.figure(figsize=(8,4))

plt.plot(np.linspace(-72,72,49,dtype=int),cwva_bin[0,:],color='k')
plt.plot(np.linspace(-72,72,49,dtype=int),cwva_bin[1,:],color='b')
plt.plot(np.linspace(-72,72,49,dtype=int),cwva_bin[2,:],color='orange')
plt.plot(np.linspace(-72,72,49,dtype=int),cwva_bin[3,:],color='r')

plt.legend(['0 < cldfrac < 0.1','0.1 < cldfrac < 0.25','0.25 < cldfrac < 0.5','0.5 < cldfrac < 1']
          ,fontsize=9)
plt.ylabel('CWV anomaly [mm]')
plt.xlabel('Lag time [hr]')
plt.grid(axis='x',linestyle='--')
#plt.title('Precipitation evolution, 10deg box, IND')

In [ ]:
def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map"""

    # Note that if base_cmap is a string or None, you can simply do
    #    return plt.cm.get_cmap(base_cmap, N)
    # The following works for string, None, or a colormap instance:

    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)

In [ ]:
# prec anomaly vs cwv anomaly
fig = plt.figure(figsize=(6,6))

plt.plot(cwva_bin[0,:],preca_bin[0,:],'-k',linewidth=0.6)
plt.scatter(cwva_bin[0,:],preca_bin[0,:],c=np.arange(-24,25),s=80
            ,cmap=discrete_cmap(6, 'RdBu'),zorder=2,edgecolor='grey')
plt.plot(cwva_bin[1,:],preca_bin[1,:],'-r',linewidth=0.6)
plt.scatter(cwva_bin[1,:],preca_bin[1,:],c=np.arange(-24,25),s=80
            ,cmap=discrete_cmap(6, 'RdBu'),zorder=2,edgecolor='grey')
plt.plot(cwva_bin[2,:],preca_bin[2,:],'-g',linewidth=0.6)
plt.scatter(cwva_bin[2,:],preca_bin[2,:],c=np.arange(-24,25),s=80
            ,cmap=discrete_cmap(6, 'RdBu'),zorder=2,edgecolor='grey')
plt.plot(cwva_bin[3,:],preca_bin[3,:],'-b',linewidth=0.6)
plt.scatter(cwva_bin[3,:],preca_bin[3,:],c=np.arange(-24,25),s=80
            ,cmap=discrete_cmap(6, 'RdBu'),zorder=2,edgecolor='grey')

In [ ]:
# open T_clim, qv_clim, RH_clim 
os.chdir('/w2-data/willytsai/ISCCP/')
T_clim = xr.open_dataset('T_clim_10deg.nc')
qv_clim = xr.open_dataset('qv_clim_10deg.nc')
RH_clim = xr.open_dataset('rh_clim_10deg.nc')

In [ ]:
T_rmseason = np.copy(T_all)

for n in range(len(time_event)):
    T_climsub = T_clim.sel(lat=slice(lat_event[n]-5,lat_event[n]+5),
                          lon=slice(lon_event[n]-5,lon_event[n]+5),month=time_event[n].month).T
    T_rmseason[n,:,:] = (T_all[n,:,:] - T_climsub.mean(axis=(1,2)).values)

In [ ]:
q_rmseason = np.copy(q_all)

for n in range(len(time_event)):
    q_climsub = qv_clim.sel(lat=slice(lat_event[n]-5,lat_event[n]+5),
                          lon=slice(lon_event[n]-5,lon_event[n]+5),month=time_event[n].month).QV
    q_rmseason[n,:,:] = q_all[n,:,:] - q_climsub.mean(axis=(1,2)).values

In [ ]:
rh_rmseason = np.copy(rh_all)

for n in range(len(time_event)):
    rh_climsub = RH_clim.sel(lat=slice(lat_event[n]-5,lat_event[n]+5),
                          lon=slice(lon_event[n]-5,lon_event[n]+5),month=time_event[n].month).RH
    rh_rmseason[n,:,:] = rh_all[n,:,:] - rh_climsub.mean(axis=(1,2)).values

In [ ]:
# os.chdir(MERRA2_3d_dir+'RH_3hr')
# files = glob('*nc4')
# data = xr.open_mfdataset(files)

In [ ]:
# %%time
# rh = data.sel(lon=slice(50,90),lat=slice(-10,10)).RH
# rh_clim = rh.groupby('time.month').mean('time').compute() # climatology of daily data

In [ ]:
# os.chdir('/w2-data/willytsai/ISCCP')
# m2data = np.loadtxt('MERRA2_modellevs.txt')
# m2lev = np.flipud(m2data[:,1])

In [ ]:
# for m in range(12):
#     plt.plot(np.nanmean(rh_clim[m,:,:,:],axis=(1,2)),m2lev,zorder=1)
# plt.ylim([1000,150])
# plt.legend(['1','2','3','4','5','6','7','8','9','10','11','12'])

In [ ]:
for m in range(12):
    plt.plot(qv_clim[m,:,:,:].mean(axis=(1,2)),lev,zorder=1)
plt.ylim([1000,150])
plt.legend(['1','2','3','4','5','6','7','8','9','10','11','12'])

In [ ]:
u = data.sel(lon=slice(50,90),lat=slice(-10,10)).U
u_clim = u.groupby('time.month').mean('time').compute() # climatology of daily data

u_rmseason = np.copy(u_all)

for n in range(len(time_event)):
    u_climsub = u_clim.sel(lat=slice(lat_event[n]-5,lat_event[n]+5),
                          lon=slice(lon_event[n]-5,lon_event[n]+5),month=time_event[n].month)
    u_rmseason[n,:] = u_all[n,:,:] - u_climsub.mean(axis=(1,2)).values

In [ ]:
for m in range(12):
    plt.plot(u_clim[m,:,:,:].mean(axis=(1,2)),lev,zorder=1)
plt.ylim([1000,150]);plt.vlines(0,ymax=1000,ymin=0,zorder=2)
plt.xlim([-10,10])
plt.legend(['1','2','3','4','5','6','7','8','9','10','11','12'])

In [ ]:
lev_model = np.loadtxt('MERRA2_modellevs.txt')[:,1]

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # mm/day
rha_bin = np.zeros((len(cldfrac_bin)-1,49,72)) # m/s
rhastd_bin = np.zeros((len(cldfrac_bin)-1,49,72))
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    rha_bin[n,:,:] = np.nanmean(rh_all[idx,:,:],axis=0)
    rhastd_bin[n,:,:] = np.nanstd(rh_all[idx,:,:],axis=0)/np.sqrt(len(idx))
    num_bin[n] = len(idx)
    
#fig=plt.figure(figsize=(8,4))

clevs = np.linspace(0.3,1,8)
plt.contourf(np.linspace(-72,72,49,dtype=int),np.flipud(lev_model),rha_bin[0,:,:].T
             ,levels=clevs,cmap='jet');
plt.colorbar();plt.ylim([1000,150]);plt.show()
plt.contourf(np.linspace(-72,72,49,dtype=int),np.flipud(lev_model),rha_bin[1,:,:].T
             ,levels=clevs,cmap='jet');
plt.colorbar();plt.ylim([1000,150]);plt.show()
plt.contourf(np.linspace(-72,72,49,dtype=int),np.flipud(lev_model),rha_bin[2,:,:].T
             ,levels=clevs,cmap='jet');
plt.colorbar();plt.ylim([1000,150]);plt.show()
plt.contourf(np.linspace(-72,72,49,dtype=int),np.flipud(lev_model),rha_bin[3,:,:].T
             ,levels=clevs,cmap='jet');
plt.colorbar();plt.ylim([1000,150]);plt.show()

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # mm/day
qa_bin = np.zeros((len(cldfrac_bin)-1,49,42)) # m/s
qastd_bin = np.zeros((len(cldfrac_bin)-1,49,42))
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    qa_bin[n,:,:] = np.nanmean(q_rmseason[idx,:,:],axis=0)
    qastd_bin[n,:,:] = np.nanstd(q_rmseason[idx,:,:],axis=0)/np.sqrt(len(idx))
    num_bin[n] = len(idx)
    
#fig=plt.figure(figsize=(8,4))

clevs = np.linspace(-1,1,21)
plt.contourf(np.linspace(-72,72,49,dtype=int),lev,qa_bin[0,:,:].T*1000,cmap='jet',
            levels=clevs
            );
plt.colorbar();plt.ylim([1000,150]);plt.show()
plt.contourf(np.linspace(-72,72,49,dtype=int),lev,qa_bin[1,:,:].T*1000,cmap='jet',
            levels=clevs
            );
plt.colorbar();plt.ylim([1000,150]);plt.show()
plt.contourf(np.linspace(-72,72,49,dtype=int),lev,qa_bin[2,:,:].T*1000,cmap='jet',
             levels=clevs
            );
plt.colorbar();plt.ylim([1000,150]);plt.show()
plt.contourf(np.linspace(-72,72,49,dtype=int),lev,qa_bin[3,:,:].T*1000,cmap='jet',
             levels=clevs
            );
plt.colorbar();plt.ylim([1000,150]);plt.show()

#plt.legend(['0 < cldfrac < 0.1','0.1 < cldfrac < 0.25','0.25 < cldfrac < 0.5','0.5 < cldfrac < 1']
#          ,fontsize=9)
#plt.ylabel('CWV anomaly [mm]')
#plt.xlabel('Lag time [hr]')
#plt.grid(axis='x',linestyle='--')
#plt.title('Precipitation evolution, 10deg box, IND')

In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # mm/day
Ta_bin = np.zeros((len(cldfrac_bin)-1,49,42)) # m/s
Tastd_bin = np.zeros((len(cldfrac_bin)-1,49,42))
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    Ta_bin[n,:,:] = np.nanmean(T_rmseason[idx,:,:],axis=0)
    Tastd_bin[n,:,:] = np.nanstd(T_rmseason[idx,:,:],axis=0)/np.sqrt(len(idx))
    num_bin[n] = len(idx)
    
#fig=plt.figure(figsize=(8,4))

clevs1 = np.linspace(-0.5,0.5,21)

plt.contourf(np.linspace(-72,72,49,dtype=int),lev,Ta_bin[0,:,:].T#,levels=clevs1
             ,cmap='RdBu_r');
cbar = plt.colorbar();plt.ylim([1000,150]); cbar.set_label('T anomaly [K]')
plt.contour(np.linspace(-72,72,49,dtype=int),lev,Ta_bin[0,:,:].T
            ,levels=[-0.3,-0.2,-0.1,0.1,0.2,0.3],colors=['k'],linewidths=0.6)
plt.ylabel('[hPa]');plt.xlabel('[hr]');plt.show()

plt.contourf(np.linspace(-72,72,49,dtype=int),lev,Ta_bin[1,:,:].T#,levels=clevs1
             ,cmap='RdBu_r');
cbar = plt.colorbar();plt.ylim([1000,150]); cbar.set_label('T anomaly [K]')
plt.contour(np.linspace(-72,72,49,dtype=int),lev,Ta_bin[1,:,:].T
            ,levels=[-0.3,-0.2,-0.1,0.1,0.2,0.3],colors=['k'],linewidths=0.6)
plt.ylabel('[hPa]');plt.xlabel('[hr]');plt.show()

plt.contourf(np.linspace(-72,72,49,dtype=int),lev,Ta_bin[2,:,:].T#,levels=clevs1
             ,cmap='RdBu_r');
cbar = plt.colorbar();plt.ylim([1000,150]); cbar.set_label('T anomaly [K]')
plt.contour(np.linspace(-72,72,49,dtype=int),lev,Ta_bin[2,:,:].T
            ,levels=[-0.3,-0.2,-0.1,0.1,0.2,0.3],colors=['k'],linewidths=0.6)
plt.ylabel('[hPa]');plt.xlabel('[hr]');plt.show()

plt.contourf(np.linspace(-72,72,49,dtype=int),lev,Ta_bin[3,:,:].T#,levels=clevs1
             ,cmap='RdBu_r');
cbar = plt.colorbar();plt.ylim([1000,150]); cbar.set_label('T anomaly [K]')
plt.contour(np.linspace(-72,72,49,dtype=int),lev,Ta_bin[3,:,:].T
            ,levels=[-0.3,-0.2,-0.1,0.1,0.2,0.3],colors=['k'],linewidths=0.6)
plt.ylabel('[hPa]');plt.xlabel('[hr]');plt.show()


In [ ]:
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # mm/day
cape_bin = np.zeros((len(cldfrac_bin)-1,49)) # m/s
capestd_bin = np.zeros((len(cldfrac_bin)-1,49))
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    cape_bin[n,:] = np.nanmean(cape_all[idx,:],axis=0)
    capestd_bin[n,:] = np.nanstd(cape_all[idx,:],axis=0)/np.sqrt(len(idx))
    num_bin[n] = len(idx)
    
fig=plt.figure(figsize=(8,4))

plt.plot(np.linspace(-72,72,49,dtype=int),cape_bin[0,:],color='k')
plt.plot(np.linspace(-72,72,49,dtype=int),cape_bin[1,:],color='b')
plt.plot(np.linspace(-72,72,49,dtype=int),cape_bin[2,:],color='orange')
plt.plot(np.linspace(-72,72,49,dtype=int),cape_bin[3,:],color='r')

plt.legend(['0 < cldfrac < 0.1','0.1 < cldfrac < 0.25','0.25 < cldfrac < 0.5','0.5 < cldfrac < 1']
          ,fontsize=9)
plt.ylabel('CWV anomaly [mm]')
plt.xlabel('Lag time [hr]')
plt.grid(axis='x',linestyle='--')
#plt.title('Precipitation evolution, 10deg box, IND')

In [ ]:
div_all.shape

In [ ]:
%%time
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # cldfrac at ref
div_regress = np.zeros((len(time_event),49,42))

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]

    for t in idx: # samples
        for k in range(div_regress.shape[2]): # vertical dims
            tmp = prec_all[t,:];tmp2 = div_all[t,:,k]
            pp = tmp[~np.isnan(tmp2)]; qq = tmp2[~np.isnan(tmp2)]  
            if len(pp) > 0: # not empty
                reg = [np.polyfit(pp,np.roll(qq,-lag),deg=1)[0] for lag in range(-24,25)]
                div_regress[t,:,k] = np.asarray(reg)
            else:
                div_regress[t,:,k] = np.nan

In [ ]:
# divergence evolution
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # mm/day
div_bin = np.zeros((len(cldfrac_bin)-1,49,42)) # m/s
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    div_bin[n,:,:] = np.nanmean(div_regress[idx,:,:],axis=0)
    num_bin[n] = len(idx)
    
fig=plt.figure(figsize=(8,4))
clevs = np.linspace(-0.6,0.6,17)
plt.contourf(np.linspace(-72,72,49,dtype=int),lev,div_bin[0,:,:].T*1e5,
             levels=clevs,cmap='PiYG',extend='both');
cbar=plt.colorbar();plt.ylim([1000,150]);cbar.set_label('s$^{-1}$/[mm/hr]');plt.show()

fig=plt.figure(figsize=(8,4))
plt.contourf(np.linspace(-72,72,49,dtype=int),lev,div_bin[1,:,:].T*1e5,
             levels=clevs,cmap='PiYG',extend='both');
cbar=plt.colorbar();plt.ylim([1000,150]);cbar.set_label('s$^{-1}$/[mm/hr]');plt.show()

fig=plt.figure(figsize=(8,4))
plt.contourf(np.linspace(-72,72,49,dtype=int),lev,div_bin[2,:,:].T*1e5,
             levels=clevs,cmap='PiYG',extend='both');
cbar=plt.colorbar();plt.ylim([1000,150]);cbar.set_label('s$^{-1}$/[mm/hr]');plt.show()

fig=plt.figure(figsize=(8,4))
plt.contourf(np.linspace(-72,72,49,dtype=int),lev,div_bin[3,:,:].T*1e5,
             levels=clevs,cmap='PiYG',extend='both');
cbar=plt.colorbar();plt.ylim([1000,150]);cbar.set_label('s$^{-1}$/[mm/hr]');plt.show()

In [ ]:
# build cloud types climatology
os.chdir(ISCCP_dir)
files = sorted(glob('*nc'))
data = xr.open_mfdataset(files)

cldamt_types = data.cldamt_types.sel(lat=slice(-10,10),lon=slice(50,90))
cldamt_cu = cldamt_types[:,0,:,:] + cldamt_types[:,3,:,:]
cldamt_st = cldamt_types[:,13,:,:] + cldamt_types[:,16,:,:]
cldamt_dc = cldamt_types[:,14,:,:] + cldamt_types[:,17,:,:]

cu_clim = cldamt_cu.groupby('time.month').mean('time').compute() # climatology of daily data
st_clim = cldamt_st.groupby('time.month').mean('time').compute()
dc_clim = cldamt_dc.groupby('time.month').mean('time').compute()

cu_rmseason = np.copy(cldamt_cu_all)
st_rmseason = np.copy(cu_rmseason)
dc_rmseason = np.copy(cu_rmseason)

In [ ]:
for n in range(len(time_event)):
    cu_climsub = cu_clim.sel(lat=slice(lat_event[n]-5,lat_event[n]+5),
                          lon=slice(lon_event[n]-5,lon_event[n]+5),month=time_event[n].month)
    st_climsub = cu_clim.sel(lat=slice(lat_event[n]-5,lat_event[n]+5),
                          lon=slice(lon_event[n]-5,lon_event[n]+5),month=time_event[n].month)
    dc_climsub = cu_clim.sel(lat=slice(lat_event[n]-5,lat_event[n]+5),
                          lon=slice(lon_event[n]-5,lon_event[n]+5),month=time_event[n].month)
    
    cu_rmseason[n,:] = cldamt_cu_all[n,:] - cu_climsub.mean(axis=(0,1)).values
    st_rmseason[n,:] = cldamt_st_all[n,:] - st_climsub.mean(axis=(0,1)).values
    dc_rmseason[n,:] = cldamt_dc_all[n,:] - dc_climsub.mean(axis=(0,1)).values

In [ ]:
%%time
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # cldfrac at ref
cldamt_cu_regress = np.zeros((len(time_event),49))
cldamt_st_regress = np.zeros((len(time_event),49))
cldamt_dc_regress = np.zeros((len(time_event),49))

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]

    for t in idx: # samples
        tmp = prec_all[t,:];tmp2 = cldamt_cu_all[t,:]
        pp = tmp[~np.isnan(tmp2)]; qq = tmp2[~np.isnan(tmp2)]  
        if len(pp) > 0: # not empty
            reg = [np.polyfit(pp,np.roll(qq,-lag),deg=1)[0] for lag in range(-24,25)]
            cldamt_cu_regress[t,:] = np.asarray(reg)
        else:
            cldamt_cu_regress[t,:] = np.nan
            
for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]

    for t in idx: # samples
        tmp = prec_all[t,:];tmp2 = cldamt_st_all[t,:]
        pp = tmp[~np.isnan(tmp2)]; qq = tmp2[~np.isnan(tmp2)]  
        if len(pp) > 0: # not empty
            reg = [np.polyfit(pp,np.roll(qq,-lag),deg=1)[0] for lag in range(-24,25)]
            cldamt_st_regress[t,:] = np.asarray(reg)
        else:
            cldamt_st_regress[t,:] = np.nan

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]

    for t in idx: # samples
        tmp = prec_all[t,:];tmp2 = cldamt_dc_all[t,:]
        pp = tmp[~np.isnan(tmp2)]; qq = tmp2[~np.isnan(tmp2)]  
        if len(pp) > 0: # not empty
            reg = [np.polyfit(pp,np.roll(qq,-lag),deg=1)[0] for lag in range(-24,25)]
            cldamt_dc_regress[t,:] = np.asarray(reg)
        else:
            cldamt_dc_regress[t,:] = np.nan

In [ ]:
# cloud type evolution
cldfrac_bin = np.array([0,0.1,0.25,0.5,1]) # mm/day
cldamt_cu_bin = np.zeros((len(cldfrac_bin)-1,49)) # m/s
cldamt_st_bin = np.zeros((len(cldfrac_bin)-1,49)) # m/s
cldamt_dc_bin = np.zeros((len(cldfrac_bin)-1,49)) # m/s
num_bin = np.copy(cldfrac_bin)*0

for n in range(len(cldfrac_bin)-1):
    idx = np.where(np.logical_and(cldfrac_all[:,24]>=cldfrac_bin[n],cldfrac_all[:,24]<cldfrac_bin[n+1]))[0]
    cldamt_cu_bin[n,:] = np.nanmean(cldamt_cu_regress[idx,:],axis=0)
    cldamt_st_bin[n,:] = np.nanmean(cldamt_st_regress[idx,:],axis=0)    
    cldamt_dc_bin[n,:] = np.nanmean(cldamt_dc_regress[idx,:],axis=0)
    num_bin[n] = len(idx)

In [ ]:
fig,ax = plt.subplots(2,2,figsize=(10,6))

ax[0,0].plot(cldamt_cu_bin[0,:],'k')
ax[0,0].plot(cldamt_st_bin[0,:],'b')
ax[0,0].plot(cldamt_dc_bin[0,:],'r')

ax[0,1].plot(cldamt_cu_bin[1,:],'k')
ax[0,1].plot(cldamt_st_bin[1,:],'b')
ax[0,1].plot(cldamt_dc_bin[1,:],'r')

ax[1,0].plot(cldamt_cu_bin[2,:],'k')
ax[1,0].plot(cldamt_st_bin[2,:],'b')
ax[1,0].plot(cldamt_dc_bin[2,:],'r')

ax[1,1].plot(cldamt_cu_bin[3,:],'k')
ax[1,1].plot(cldamt_st_bin[3,:],'b')
ax[1,1].plot(cldamt_dc_bin[3,:],'r')

plt.show()